*Copyright 2024 The Penzai Authors.*

*Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at*

> http://www.apache.org/licenses/LICENSE-2.0

*Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
See the License for the specific language governing permissions and
limitations under the License.*

---

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-deepmind/penzai/blob/main/notebooks/treescope_arrayviz.ipynb) [![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/google-deepmind/penzai/blob/main/notebooks/treescope_arrayviz.ipynb)

# Visualizing NDArrays with Treescope

High-dimensional NDArray (or tensor) data is common in many machine learning settings, but most plotting libraries are designed for either 2D image data or 1D time series data. Penzai's pretty printer (`treescope`) includes a powerful arbitrarily-high-dimensional-array visualizer designed to make it easy to quickly summarize NDArrays without having to write manual plotting logic.

## Setup

To run this notebook, you need a Python environment with `penzai` and its dependencies installed.

In Colab or Kaggle, you can install it using the following command:

In [1]:
try:
  import penzai
except ImportError:
  !pip install penzai[notebook]

In [2]:
from __future__ import annotations
from typing import Any

import jax
import jax.numpy as jnp
import numpy as np

import IPython

In [3]:
import penzai
from penzai import pz

## Visualizing NDArrays with `pz.ts.render_array`

Treescope includes a powerful array renderer, defined in `penzai.treescope.arrayviz` and aliased to `pz.ts.render_array` for easy use. It is designed to make faceted, interactive visualizations of $N$-dimensional arrays.

### Visualizing numeric data and customizing colormaps

Arrays can be directly rendered using default settings by passing them to `pz.ts.render_array`:

In [4]:
help(pz.ts.render_array)

Help on function render_array in module penzai.treescope.arrayviz.arrayviz:

render_array(array: 'named_axes.NamedArray | named_axes.NamedArrayView | np.ndarray | jax.Array', *, columns: 'Sequence[named_axes.AxisName | int]' = (), rows: 'Sequence[named_axes.AxisName | int]' = (), sliders: 'Sequence[named_axes.AxisName | int]' = (), valid_mask: 'named_axes.NamedArray | named_axes.NamedArrayView | np.ndarray | jax.Array | None' = None, continuous: "bool | Literal['auto']" = 'auto', around_zero: "bool | Literal['auto']" = 'auto', vmax: 'float | None' = None, vmin: 'float | None' = None, trim_outliers: 'bool' = True, dynamic_colormap: "bool | Literal['auto']" = 'auto', colormap: 'list[tuple[int, int, int]] | None' = None, truncate: 'bool' = False, maximum_size: 'int' = 10000, cutoff_size_per_axis: 'int' = 512, minimum_edge_items: 'int' = 5, axis_item_labels: 'dict[named_axes.AxisName | int, list[str]] | None' = None, value_item_labels: 'dict[int, str] | None' = None, axis_labels: 'dict[nam

In [5]:
my_array = np.cos(np.arange(300).reshape((10,30)) * 0.2)

pz.ts.render_array(my_array)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACAP6Xlej+nyms/MklTP19bMj9AUQo/9Ya5Pt4LLj7WM++8qqdovjMR1b4CqBa/1sU8vxBdW798NXG/JnB9vz+Qf78WgHe/9pFlv9x8Sr8wVSe/dwP7vrxanb45sOW9qzKzPSw8kT5r4e8+O3siP3yLRj9osWI/uM11P2Idfz9eQX4/cUJzP9WQXj+9/0A/6bwbP0aJ4D4hpYA+XABdPfb9FL61pa2+GvAEv5nALb/Lo0+/1T9pv1qPeb/i63+/gRR8v2swbr9kzVa/Hdo2v6qcD78MS8W+a/5GvnIFkTuC4E8+o3fJPm57ET9Dbjg/0AZYP50Cbz8Zd3w/8tp/P44LeT9vTmg/aU5OP9kULD8I/wI/kWCpPtcEDD7ymIC9yAaFvquZ5L6+hx2/6HpCvzqtX7+59HO/bYJ+v6Dqfr8sKXW/oaFhv08bRb9XuSC/EN7rvjbijL4JIqG9R7H3PSqpoT4e9P4+hAopP8HdSz9bkGY/1xF4P4yvfz+/G30/v3BwP9QvWj8fPDs/gtEUPyLw0D6y0V8+0LamPK34Nr5pvr2+/DcMv7X5M7+djlS/L6psv19We79v/X+/4m96v2Xmar+K/1G/dbkwv79nCL/IS7W+kRslvgjWGz34eHE+/i3ZPn57GD/ESz4/5YVcP2L1cT94v30/0mt/P1/pdj/2jmQ/5xdJP4GcJT/IDfc+JQmZPpCq0z21P8W9OJOVvvbf877eOSS/rvdHv6S8Y79XbXa/Bkt/vzb7fb9Ji3K/+m9dv7OAP7/27hm/YXTcvs2BeL5kyji9GPQdPl3nsT6B3gY/3WgvPwL1UD+OLGo/JQ56P7D3fz/arHs/cVltP6qPVT9MQjU/BbsNP4EawT5XGD4+t4ZZvG2+WL4uoM2+R1cTv7f+Ob/nO1m/BdBvv6HUfL/gxH+/w4J4v19YZ7/k9Ey/pWUqv1YLAb8IGKW+6ggDviOvkj3DZYk+eqXoPmlPHz8t8kM/IsVgPxyidD9hvn4/wrJ+P7V/dD9TjWA/LqdDPzj0Hj/71ec+bIWIPisOjz2v1gS+W/SlvtNvAb90vCq/kDpNvyCKZ7+dnni/t8l/v0TCfL8vp2+/Of5Yv6auOb8F+BK/4MrMvj33Vr7jbDy80uE/Pg7ywT7oGw4/bpQ1P8XPVT/3hG0/D8J7P7v1fz8a9Xk/bf1pP6yxUD8BFC8/gHsGP/kMsT70Jxw+BQ9AvW5Fer5tRt2+6ksav+DNP79Nql2/b7Byv7MJfr9GQn+/tE12v2CHY7/qrke/gOAjvyUT8759tJS+qKDBvcpI1z1E55k+nNn3Pjb1JT/dX0k/TsNkPwMIdz+Lc38/9a99P0PPcT+uSlw/0f09P+wdGD8QW9g+WrRvPriQFD0O5ya+cyW2vjPKCL+cDTG/CEJSv5UUa7/rh3q/XP5/vydAe7+0fWy/pk1Uv9imM7+I1gu/Fua8vnMuNb4sQ7U8/5dhPpvE0T4rMBU/b4s7P6JsWj+dmHA/Fy19P62pfz/99Hc/rF1mP0OXSz8Gsyg/Gyr+Pi/MoD63FPQ9YcKkvf3Bjb6erOy+4hMhv4NlRb+I2GG/l0p1vzn1fr/HdX6/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 30}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 30", "start": 0, "end": 30}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -1.0, "max": 1.0, "dynamic": true, "rawMinAbs": 0.004425697959959507, "rawMaxAbs": 1.0, "cmapData": [[96, 14, 34], [134, 14, 41], [167, 36, 36], [186, 72, 46], [198, 107, 77], [208, 139, 115], [218, 171, 155], [228, 203, 196], [241, 236, 235], [202, 212, 216], [161, 190, 200], [117, 170, 190], [75, 148, 186], [38, 123, 186], [12, 94, 190], [41, 66, 162], [37, 47, 111]]}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": ", "}, {"type": "index", "axis": "a1", "skip_start": 30, "skip_end": 30}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

Things to notice:
- The visualization is interactive! (Try zooming in and out, hovering over the array to inspect individual elements, or clicking to remember a particular element.)
- The shape of the array can be read off by looking at the axis labels.
- Pixels are *always* square in arrayviz renderings. (In fact, they are always exactly 7 pixels by 7 pixels at zoom level 1.)

The default rendering strategy uses a diverging colormap centered at zero, with blue for positive numbers and red for negative ones, to show you the absolute magnitude and sign of the array. You can toggle to a relative mode by passing the argument `around_zero=False`:

In [6]:
pz.ts.render_array(my_array, around_zero=False)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Linear colormap from -0.999975 to 1. Hover/click for array data.", "arrayBase64": "AACAP6Xlej+nyms/MklTP19bMj9AUQo/9Ya5Pt4LLj7WM++8qqdovjMR1b4CqBa/1sU8vxBdW798NXG/JnB9vz+Qf78WgHe/9pFlv9x8Sr8wVSe/dwP7vrxanb45sOW9qzKzPSw8kT5r4e8+O3siP3yLRj9osWI/uM11P2Idfz9eQX4/cUJzP9WQXj+9/0A/6bwbP0aJ4D4hpYA+XABdPfb9FL61pa2+GvAEv5nALb/Lo0+/1T9pv1qPeb/i63+/gRR8v2swbr9kzVa/Hdo2v6qcD78MS8W+a/5GvnIFkTuC4E8+o3fJPm57ET9Dbjg/0AZYP50Cbz8Zd3w/8tp/P44LeT9vTmg/aU5OP9kULD8I/wI/kWCpPtcEDD7ymIC9yAaFvquZ5L6+hx2/6HpCvzqtX7+59HO/bYJ+v6Dqfr8sKXW/oaFhv08bRb9XuSC/EN7rvjbijL4JIqG9R7H3PSqpoT4e9P4+hAopP8HdSz9bkGY/1xF4P4yvfz+/G30/v3BwP9QvWj8fPDs/gtEUPyLw0D6y0V8+0LamPK34Nr5pvr2+/DcMv7X5M7+djlS/L6psv19We79v/X+/4m96v2Xmar+K/1G/dbkwv79nCL/IS7W+kRslvgjWGz34eHE+/i3ZPn57GD/ESz4/5YVcP2L1cT94v30/0mt/P1/pdj/2jmQ/5xdJP4GcJT/IDfc+JQmZPpCq0z21P8W9OJOVvvbf877eOSS/rvdHv6S8Y79XbXa/Bkt/vzb7fb9Ji3K/+m9dv7OAP7/27hm/YXTcvs2BeL5kyji9GPQdPl3nsT6B3gY/3WgvPwL1UD+OLGo/JQ56P7D3fz/arHs/cVltP6qPVT9MQjU/BbsNP4EawT5XGD4+t4ZZvG2+WL4uoM2+R1cTv7f+Ob/nO1m/BdBvv6HUfL/gxH+/w4J4v19YZ7/k9Ey/pWUqv1YLAb8IGKW+6ggDviOvkj3DZYk+eqXoPmlPHz8t8kM/IsVgPxyidD9hvn4/wrJ+P7V/dD9TjWA/LqdDPzj0Hj/71ec+bIWIPisOjz2v1gS+W/SlvtNvAb90vCq/kDpNvyCKZ7+dnni/t8l/v0TCfL8vp2+/Of5Yv6auOb8F+BK/4MrMvj33Vr7jbDy80uE/Pg7ywT7oGw4/bpQ1P8XPVT/3hG0/D8J7P7v1fz8a9Xk/bf1pP6yxUD8BFC8/gHsGP/kMsT70Jxw+BQ9AvW5Fer5tRt2+6ksav+DNP79Nql2/b7Byv7MJfr9GQn+/tE12v2CHY7/qrke/gOAjvyUT8759tJS+qKDBvcpI1z1E55k+nNn3Pjb1JT/dX0k/TsNkPwMIdz+Lc38/9a99P0PPcT+uSlw/0f09P+wdGD8QW9g+WrRvPriQFD0O5ya+cyW2vjPKCL+cDTG/CEJSv5UUa7/rh3q/XP5/vydAe7+0fWy/pk1Uv9imM7+I1gu/Fua8vnMuNb4sQ7U8/5dhPpvE0T4rMBU/b4s7P6JsWj+dmHA/Fy19P62pfz/99Hc/rF1mP0OXSz8Gsyg/Gyr+Pi/MoD63FPQ9YcKkvf3Bjb6erOy+4hMhv4NlRb+I2GG/l0p1vzn1fr/HdX6/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 30}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 30", "start": 0, "end": 30}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -0.9999749660491943, "max": 1.0, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": [[68, 1, 84], [72, 20, 103], [72, 38, 119], [69, 55, 129], [63, 71, 136], [57, 85, 140], [50, 100, 142], [45, 113, 142], [40, 125, 142], [35, 138, 141], [31, 150, 139], [32, 163, 134], [41, 175, 127], [59, 187, 117], [86, 198, 103], [115, 208, 86], [149, 216, 64], [184, 222, 41], [221, 227, 24], [253, 231, 37]]}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": ", "}, {"type": "index", "axis": "a1", "skip_start": 30, "skip_end": 30}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

You can also customize the upper and lower bounds of the colormap by passing `vmin` and/or `vmax`:

In [7]:
pz.ts.render_array(my_array, vmax=0.7)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Linear colormap from -0.7 to 0.7. Hover/click for array data.", "arrayBase64": "AACAP6Xlej+nyms/MklTP19bMj9AUQo/9Ya5Pt4LLj7WM++8qqdovjMR1b4CqBa/1sU8vxBdW798NXG/JnB9vz+Qf78WgHe/9pFlv9x8Sr8wVSe/dwP7vrxanb45sOW9qzKzPSw8kT5r4e8+O3siP3yLRj9osWI/uM11P2Idfz9eQX4/cUJzP9WQXj+9/0A/6bwbP0aJ4D4hpYA+XABdPfb9FL61pa2+GvAEv5nALb/Lo0+/1T9pv1qPeb/i63+/gRR8v2swbr9kzVa/Hdo2v6qcD78MS8W+a/5GvnIFkTuC4E8+o3fJPm57ET9Dbjg/0AZYP50Cbz8Zd3w/8tp/P44LeT9vTmg/aU5OP9kULD8I/wI/kWCpPtcEDD7ymIC9yAaFvquZ5L6+hx2/6HpCvzqtX7+59HO/bYJ+v6Dqfr8sKXW/oaFhv08bRb9XuSC/EN7rvjbijL4JIqG9R7H3PSqpoT4e9P4+hAopP8HdSz9bkGY/1xF4P4yvfz+/G30/v3BwP9QvWj8fPDs/gtEUPyLw0D6y0V8+0LamPK34Nr5pvr2+/DcMv7X5M7+djlS/L6psv19We79v/X+/4m96v2Xmar+K/1G/dbkwv79nCL/IS7W+kRslvgjWGz34eHE+/i3ZPn57GD/ESz4/5YVcP2L1cT94v30/0mt/P1/pdj/2jmQ/5xdJP4GcJT/IDfc+JQmZPpCq0z21P8W9OJOVvvbf877eOSS/rvdHv6S8Y79XbXa/Bkt/vzb7fb9Ji3K/+m9dv7OAP7/27hm/YXTcvs2BeL5kyji9GPQdPl3nsT6B3gY/3WgvPwL1UD+OLGo/JQ56P7D3fz/arHs/cVltP6qPVT9MQjU/BbsNP4EawT5XGD4+t4ZZvG2+WL4uoM2+R1cTv7f+Ob/nO1m/BdBvv6HUfL/gxH+/w4J4v19YZ7/k9Ey/pWUqv1YLAb8IGKW+6ggDviOvkj3DZYk+eqXoPmlPHz8t8kM/IsVgPxyidD9hvn4/wrJ+P7V/dD9TjWA/LqdDPzj0Hj/71ec+bIWIPisOjz2v1gS+W/SlvtNvAb90vCq/kDpNvyCKZ7+dnni/t8l/v0TCfL8vp2+/Of5Yv6auOb8F+BK/4MrMvj33Vr7jbDy80uE/Pg7ywT7oGw4/bpQ1P8XPVT/3hG0/D8J7P7v1fz8a9Xk/bf1pP6yxUD8BFC8/gHsGP/kMsT70Jxw+BQ9AvW5Fer5tRt2+6ksav+DNP79Nql2/b7Byv7MJfr9GQn+/tE12v2CHY7/qrke/gOAjvyUT8759tJS+qKDBvcpI1z1E55k+nNn3Pjb1JT/dX0k/TsNkPwMIdz+Lc38/9a99P0PPcT+uSlw/0f09P+wdGD8QW9g+WrRvPriQFD0O5ya+cyW2vjPKCL+cDTG/CEJSv5UUa7/rh3q/XP5/vydAe7+0fWy/pk1Uv9imM7+I1gu/Fua8vnMuNb4sQ7U8/5dhPpvE0T4rMBU/b4s7P6JsWj+dmHA/Fy19P62pfz/99Hc/rF1mP0OXSz8Gsyg/Gyr+Pi/MoD63FPQ9YcKkvf3Bjb6erOy+4hMhv4NlRb+I2GG/l0p1vzn1fr/HdX6/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 30}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 30", "start": 0, "end": 30}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -0.699999988079071, "max": 0.699999988079071, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": [[96, 14, 34], [134, 14, 41], [167, 36, 36], [186, 72, 46], [198, 107, 77], [208, 139, 115], [218, 171, 155], [228, 203, 196], [241, 236, 235], [202, 212, 216], [161, 190, 200], [117, 170, 190], [75, 148, 186], [38, 123, 186], [12, 94, 190], [41, 66, 162], [37, 47, 111]]}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": ", "}, {"type": "index", "axis": "a1", "skip_start": 30, "skip_end": 30}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

In this case, the array has values outside of our specified colormap bounds; those out-of-bounds values are rendered with "+" and "-" to indicate that they've been clipped.

Since we didn't pass `around_zero=False`, it automatically set `vmin` to `-vmax` for us. You can choose to set both explicitly too:

In [8]:
pz.ts.render_array(my_array, vmin=-0.1, vmax=0.7)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Linear colormap from -0.1 to 0.7. Hover/click for array data.", "arrayBase64": "AACAP6Xlej+nyms/MklTP19bMj9AUQo/9Ya5Pt4LLj7WM++8qqdovjMR1b4CqBa/1sU8vxBdW798NXG/JnB9vz+Qf78WgHe/9pFlv9x8Sr8wVSe/dwP7vrxanb45sOW9qzKzPSw8kT5r4e8+O3siP3yLRj9osWI/uM11P2Idfz9eQX4/cUJzP9WQXj+9/0A/6bwbP0aJ4D4hpYA+XABdPfb9FL61pa2+GvAEv5nALb/Lo0+/1T9pv1qPeb/i63+/gRR8v2swbr9kzVa/Hdo2v6qcD78MS8W+a/5GvnIFkTuC4E8+o3fJPm57ET9Dbjg/0AZYP50Cbz8Zd3w/8tp/P44LeT9vTmg/aU5OP9kULD8I/wI/kWCpPtcEDD7ymIC9yAaFvquZ5L6+hx2/6HpCvzqtX7+59HO/bYJ+v6Dqfr8sKXW/oaFhv08bRb9XuSC/EN7rvjbijL4JIqG9R7H3PSqpoT4e9P4+hAopP8HdSz9bkGY/1xF4P4yvfz+/G30/v3BwP9QvWj8fPDs/gtEUPyLw0D6y0V8+0LamPK34Nr5pvr2+/DcMv7X5M7+djlS/L6psv19We79v/X+/4m96v2Xmar+K/1G/dbkwv79nCL/IS7W+kRslvgjWGz34eHE+/i3ZPn57GD/ESz4/5YVcP2L1cT94v30/0mt/P1/pdj/2jmQ/5xdJP4GcJT/IDfc+JQmZPpCq0z21P8W9OJOVvvbf877eOSS/rvdHv6S8Y79XbXa/Bkt/vzb7fb9Ji3K/+m9dv7OAP7/27hm/YXTcvs2BeL5kyji9GPQdPl3nsT6B3gY/3WgvPwL1UD+OLGo/JQ56P7D3fz/arHs/cVltP6qPVT9MQjU/BbsNP4EawT5XGD4+t4ZZvG2+WL4uoM2+R1cTv7f+Ob/nO1m/BdBvv6HUfL/gxH+/w4J4v19YZ7/k9Ey/pWUqv1YLAb8IGKW+6ggDviOvkj3DZYk+eqXoPmlPHz8t8kM/IsVgPxyidD9hvn4/wrJ+P7V/dD9TjWA/LqdDPzj0Hj/71ec+bIWIPisOjz2v1gS+W/SlvtNvAb90vCq/kDpNvyCKZ7+dnni/t8l/v0TCfL8vp2+/Of5Yv6auOb8F+BK/4MrMvj33Vr7jbDy80uE/Pg7ywT7oGw4/bpQ1P8XPVT/3hG0/D8J7P7v1fz8a9Xk/bf1pP6yxUD8BFC8/gHsGP/kMsT70Jxw+BQ9AvW5Fer5tRt2+6ksav+DNP79Nql2/b7Byv7MJfr9GQn+/tE12v2CHY7/qrke/gOAjvyUT8759tJS+qKDBvcpI1z1E55k+nNn3Pjb1JT/dX0k/TsNkPwMIdz+Lc38/9a99P0PPcT+uSlw/0f09P+wdGD8QW9g+WrRvPriQFD0O5ya+cyW2vjPKCL+cDTG/CEJSv5UUa7/rh3q/XP5/vydAe7+0fWy/pk1Uv9imM7+I1gu/Fua8vnMuNb4sQ7U8/5dhPpvE0T4rMBU/b4s7P6JsWj+dmHA/Fy19P62pfz/99Hc/rF1mP0OXSz8Gsyg/Gyr+Pi/MoD63FPQ9YcKkvf3Bjb6erOy+4hMhv4NlRb+I2GG/l0p1vzn1fr/HdX6/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 30}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 30", "start": 0, "end": 30}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -0.1, "max": 0.7, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": [[68, 1, 84], [72, 20, 103], [72, 38, 119], [69, 55, 129], [63, 71, 136], [57, 85, 140], [50, 100, 142], [45, 113, 142], [40, 125, 142], [35, 138, 141], [31, 150, 139], [32, 163, 134], [41, 175, 127], [59, 187, 117], [86, 198, 103], [115, 208, 86], [149, 216, 64], [184, 222, 41], [221, 227, 24], [253, 231, 37]]}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": ", "}, {"type": "index", "axis": "a1", "skip_start": 30, "skip_end": 30}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

If you want to customize the way colors are rendered, you can pass a custom colormap as a list of (R, G, B) color tuples:

In [9]:
import palettable
pz.ts.render_array(my_array, colormap=palettable.matplotlib.Inferno_20.colors)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACAP6Xlej+nyms/MklTP19bMj9AUQo/9Ya5Pt4LLj7WM++8qqdovjMR1b4CqBa/1sU8vxBdW798NXG/JnB9vz+Qf78WgHe/9pFlv9x8Sr8wVSe/dwP7vrxanb45sOW9qzKzPSw8kT5r4e8+O3siP3yLRj9osWI/uM11P2Idfz9eQX4/cUJzP9WQXj+9/0A/6bwbP0aJ4D4hpYA+XABdPfb9FL61pa2+GvAEv5nALb/Lo0+/1T9pv1qPeb/i63+/gRR8v2swbr9kzVa/Hdo2v6qcD78MS8W+a/5GvnIFkTuC4E8+o3fJPm57ET9Dbjg/0AZYP50Cbz8Zd3w/8tp/P44LeT9vTmg/aU5OP9kULD8I/wI/kWCpPtcEDD7ymIC9yAaFvquZ5L6+hx2/6HpCvzqtX7+59HO/bYJ+v6Dqfr8sKXW/oaFhv08bRb9XuSC/EN7rvjbijL4JIqG9R7H3PSqpoT4e9P4+hAopP8HdSz9bkGY/1xF4P4yvfz+/G30/v3BwP9QvWj8fPDs/gtEUPyLw0D6y0V8+0LamPK34Nr5pvr2+/DcMv7X5M7+djlS/L6psv19We79v/X+/4m96v2Xmar+K/1G/dbkwv79nCL/IS7W+kRslvgjWGz34eHE+/i3ZPn57GD/ESz4/5YVcP2L1cT94v30/0mt/P1/pdj/2jmQ/5xdJP4GcJT/IDfc+JQmZPpCq0z21P8W9OJOVvvbf877eOSS/rvdHv6S8Y79XbXa/Bkt/vzb7fb9Ji3K/+m9dv7OAP7/27hm/YXTcvs2BeL5kyji9GPQdPl3nsT6B3gY/3WgvPwL1UD+OLGo/JQ56P7D3fz/arHs/cVltP6qPVT9MQjU/BbsNP4EawT5XGD4+t4ZZvG2+WL4uoM2+R1cTv7f+Ob/nO1m/BdBvv6HUfL/gxH+/w4J4v19YZ7/k9Ey/pWUqv1YLAb8IGKW+6ggDviOvkj3DZYk+eqXoPmlPHz8t8kM/IsVgPxyidD9hvn4/wrJ+P7V/dD9TjWA/LqdDPzj0Hj/71ec+bIWIPisOjz2v1gS+W/SlvtNvAb90vCq/kDpNvyCKZ7+dnni/t8l/v0TCfL8vp2+/Of5Yv6auOb8F+BK/4MrMvj33Vr7jbDy80uE/Pg7ywT7oGw4/bpQ1P8XPVT/3hG0/D8J7P7v1fz8a9Xk/bf1pP6yxUD8BFC8/gHsGP/kMsT70Jxw+BQ9AvW5Fer5tRt2+6ksav+DNP79Nql2/b7Byv7MJfr9GQn+/tE12v2CHY7/qrke/gOAjvyUT8759tJS+qKDBvcpI1z1E55k+nNn3Pjb1JT/dX0k/TsNkPwMIdz+Lc38/9a99P0PPcT+uSlw/0f09P+wdGD8QW9g+WrRvPriQFD0O5ya+cyW2vjPKCL+cDTG/CEJSv5UUa7/rh3q/XP5/vydAe7+0fWy/pk1Uv9imM7+I1gu/Fua8vnMuNb4sQ7U8/5dhPpvE0T4rMBU/b4s7P6JsWj+dmHA/Fy19P62pfz/99Hc/rF1mP0OXSz8Gsyg/Gyr+Pi/MoD63FPQ9YcKkvf3Bjb6erOy+4hMhv4NlRb+I2GG/l0p1vzn1fr/HdX6/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 30}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 30", "start": 0, "end": 30}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -1.0, "max": 1.0, "dynamic": true, "rawMinAbs": 0.004425697959959507, "rawMaxAbs": 1.0, "cmapData": [[0, 0, 4], [8, 5, 29], [25, 12, 62], [47, 10, 91], [71, 11, 106], [92, 18, 110], [114, 26, 110], [135, 33, 107], [155, 41, 100], [177, 50, 90], [196, 60, 78], [215, 75, 63], [229, 92, 48], [240, 111, 32], [248, 135, 14], [252, 159, 7], [251, 186, 31], [247, 211, 64], [241, 237, 113], [252, 255, 164]]}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": ", "}, {"type": "index", "axis": "a1", "skip_start": 30, "skip_end": 30}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

In [10]:
pz.ts.render_array(my_array, colormap=palettable.cmocean.sequential.Speed_20.colors)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACAP6Xlej+nyms/MklTP19bMj9AUQo/9Ya5Pt4LLj7WM++8qqdovjMR1b4CqBa/1sU8vxBdW798NXG/JnB9vz+Qf78WgHe/9pFlv9x8Sr8wVSe/dwP7vrxanb45sOW9qzKzPSw8kT5r4e8+O3siP3yLRj9osWI/uM11P2Idfz9eQX4/cUJzP9WQXj+9/0A/6bwbP0aJ4D4hpYA+XABdPfb9FL61pa2+GvAEv5nALb/Lo0+/1T9pv1qPeb/i63+/gRR8v2swbr9kzVa/Hdo2v6qcD78MS8W+a/5GvnIFkTuC4E8+o3fJPm57ET9Dbjg/0AZYP50Cbz8Zd3w/8tp/P44LeT9vTmg/aU5OP9kULD8I/wI/kWCpPtcEDD7ymIC9yAaFvquZ5L6+hx2/6HpCvzqtX7+59HO/bYJ+v6Dqfr8sKXW/oaFhv08bRb9XuSC/EN7rvjbijL4JIqG9R7H3PSqpoT4e9P4+hAopP8HdSz9bkGY/1xF4P4yvfz+/G30/v3BwP9QvWj8fPDs/gtEUPyLw0D6y0V8+0LamPK34Nr5pvr2+/DcMv7X5M7+djlS/L6psv19We79v/X+/4m96v2Xmar+K/1G/dbkwv79nCL/IS7W+kRslvgjWGz34eHE+/i3ZPn57GD/ESz4/5YVcP2L1cT94v30/0mt/P1/pdj/2jmQ/5xdJP4GcJT/IDfc+JQmZPpCq0z21P8W9OJOVvvbf877eOSS/rvdHv6S8Y79XbXa/Bkt/vzb7fb9Ji3K/+m9dv7OAP7/27hm/YXTcvs2BeL5kyji9GPQdPl3nsT6B3gY/3WgvPwL1UD+OLGo/JQ56P7D3fz/arHs/cVltP6qPVT9MQjU/BbsNP4EawT5XGD4+t4ZZvG2+WL4uoM2+R1cTv7f+Ob/nO1m/BdBvv6HUfL/gxH+/w4J4v19YZ7/k9Ey/pWUqv1YLAb8IGKW+6ggDviOvkj3DZYk+eqXoPmlPHz8t8kM/IsVgPxyidD9hvn4/wrJ+P7V/dD9TjWA/LqdDPzj0Hj/71ec+bIWIPisOjz2v1gS+W/SlvtNvAb90vCq/kDpNvyCKZ7+dnni/t8l/v0TCfL8vp2+/Of5Yv6auOb8F+BK/4MrMvj33Vr7jbDy80uE/Pg7ywT7oGw4/bpQ1P8XPVT/3hG0/D8J7P7v1fz8a9Xk/bf1pP6yxUD8BFC8/gHsGP/kMsT70Jxw+BQ9AvW5Fer5tRt2+6ksav+DNP79Nql2/b7Byv7MJfr9GQn+/tE12v2CHY7/qrke/gOAjvyUT8759tJS+qKDBvcpI1z1E55k+nNn3Pjb1JT/dX0k/TsNkPwMIdz+Lc38/9a99P0PPcT+uSlw/0f09P+wdGD8QW9g+WrRvPriQFD0O5ya+cyW2vjPKCL+cDTG/CEJSv5UUa7/rh3q/XP5/vydAe7+0fWy/pk1Uv9imM7+I1gu/Fua8vnMuNb4sQ7U8/5dhPpvE0T4rMBU/b4s7P6JsWj+dmHA/Fy19P62pfz/99Hc/rF1mP0OXSz8Gsyg/Gyr+Pi/MoD63FPQ9YcKkvf3Bjb6erOy+4hMhv4NlRb+I2GG/l0p1vzn1fr/HdX6/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 30}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 30", "start": 0, "end": 30}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -1.0, "max": 1.0, "dynamic": true, "rawMinAbs": 0.004425697959959507, "rawMaxAbs": 1.0, "cmapData": [[255, 253, 205], [246, 237, 177], [237, 221, 147], [227, 207, 119], [213, 194, 89], [197, 184, 63], [177, 174, 39], [155, 166, 20], [133, 159, 7], [108, 150, 7], [84, 142, 17], [59, 132, 28], [37, 122, 36], [19, 111, 41], [11, 99, 44], [15, 86, 44], [21, 73, 42], [25, 61, 36], [25, 47, 28], [23, 35, 19]]}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": ", "}, {"type": "index", "axis": "a1", "skip_start": 30, "skip_end": 30}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

### Visualizing high-dimensional arrays and NamedArrays

So far we've been looking at an array with two axes, but arrayviz works out-of-the-box with arbitrarily high-dimensional arrays as well:

In [11]:
my_4d_array = np.cos(np.arange(5*6*7*8).reshape((5,6,7,8)) * 0.1)
pz.ts.render_array(my_4d_array)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACAP5i4fj+l5Xo/75B0P6fKaz9AqWA/MklTP7PMQz9fWzI/1SEfP0BRCj/APeg+9Ya5Ppv1iD7eCy4+qt6QPdYz77zQ7wO+qqdovjaGpb4zEdW+mD0BvwKoFr8RkSq/1sU8v78XTb8QXVu/RnFnv3w1cb+3kHi/JnB9v1LHf78/kH+/ect8vxaAd7+gu2+/9pFlvxUdWb/cfEq/s9Y5vzBVJ7+qJxO/dwP7vo01zb68Wp2+29pXvjmw5b3S+Uq8qzKzPRn9Pj4sPJE+TIbBPmvh7z566w0/O3siP2JrNT98i0Y/va9VP2ixYj86b20/uM11P3u3ez9iHX8/vPZ/P15Bfj+mAXo/cUJzPwQVaj/VkF4/XNNQP73/QD9zPi8/6bwbPwStBj9GieA+L3qxPiGlgD4KDh0+XABdPbpsPL32/RS+o2N5vrWlrb5t3dy+GvAEv3QdGr+ZwC2/Tqc/v8ujT78pjV2/1T9pv+Kdcr9aj3m/ewJ+v+Lrf7+tRn+/gRR8v4tddr9rMG6/CKJjv2TNVr9R00e/Hdo2vzMNJL+qnA+/lHnzvgxLxb7eI5W+a/5GvjRww71yBZE7s3nVPYLgTz45eJk+o3fJPrlz9z5uexE/4MglP0NuOD/nO0k/0AZYPyipZD+dAm8/uPh2Pxl3fD+1b38/8tp/P723fT+OC3k/WeJxP29OaD9PaFw/aU5OP88kPj/ZFCw/uEwYPwj/Aj+MxNg+kWCpPq+WcD7XBAw+ZDMYPfKYgL1UASa+yAaFvqK4tb6rmeS+/ZgIv76HHb+O4zC/6HpCv88gUr86rV+/g/1qv7n0c7/te3q/bYJ+v+z9f7+g6n6/Skt7vywpdb/4k2y/oaFhvyduVL9PG0W/S9Azv1e5IL9GBwy/EN7rvkRSvb424oy+lRM2vgkiob0C/a08R7H3Pd+0YD4qqaE+ZFrRPh70/j7aABU/hAopP8tjOz/B3Us/QU5aP1uQZj+0hHA/1xF4P3IkfT+Mr38/o6x/P78bfT9wA3g/v3BwPwl3Zj/UL1o/iLpLPx88Oz/J3ig/gtEUPyOP/j4i8NA+sTqhPrLRXz4F4/U90LamPDnyor2t+Da+HVKNvmm+vb5dRey+/DcMv6DmIL+1+TO/bkBFv52OVL8avWG/L6psv+g5db9fVnu/8+9+v2/9f78hfH6/4m96vw7jc79l5mq/5ZBfv4r/Ub8JVUK/dbkwv9hZHb+/Zwi/ezHkvshLtb5aloS+kRslvmiQfb0I1hs9d+sMPvh4cT5pzqk+/i3ZPgcxAz9+exg/7D8sP8RLPj/acE4/5YVcP91maD9i9XE/ARl5P3i/fT/g3H8/0mt/P29tfD9f6XY/ve1uP/aOZD+P51c/5xdJP+BFOD+BnCU/hUsRP8gN9z6YDMk+JQmZPoL8Tj6QqtM9NtZnO7U/xb3J4ke+OJOVvnO2xb723/O+1swPv945JL/VAje/rvdHvwntVr+kvGO/uUVuv1dtdr+gHny/Bkt/v2rqf782+32/WoJ5v0mLcr/SJ2m/+m9dv7qBT7+zgD+/1ZUtv/buGb9YvgS/YXTcvi04rb7NgXi+mRcUvmTKOL1IomA9GPQdPswVgT5d57E+4PHgPoHeBj8Y6xs/3WgvP/YlQT8C9VA/kq1eP44saj+NVHM/JQ56PyBIfj+w938/hRh/P9qsez9uvXU/cVltP1eWYj+qj1U/uWZGP0xCNT88TiI/BbsNP4x67z6BGsE+hsyQPlcYPj7JYrE9t4ZZvOp+571tvli+f8mdvi6gzb7naPu+R1cTvzmBJ7+3/jm/dKBKv+c7Wb+yq2W/BdBvv++Od7+h1Hy/npN/v+DEf7/pZ32/w4J4v/Yhcb9fWGe/CT9bv+T0TL+Anjy/pWUqv+94Fr9WCwG/U6fUvggYpb7oxGe+6ggDvgLu57wjr5I9SfEuPsNliT5w87k+eqXoPjiCCj9pTx8/HIUyPy3yQz8KalM/IsVgP0zhaz8conQ/L/F6P2G+fj/5/38/wrJ+Pw7aej+1f3Q/97NrP1ONYD9PKFM/LqdDP5kxMj849B4/QiAKP/vV5z5wGrk+bIWIPmkmLT4rDo89nnn2vK/WBL5himm+W/Slvgh71b7TbwG/DtcWv3S8Kr8i7Ty/kDpNvwt7W78gime/9Uhxv52eeL9WeH2/t8l/v9OMf79Ewny/MHF3vy+nb78ueGW/Of5YvzpZSr+mrjm/HyknvwX4Er/7nfq+4MrMvvHrnL4991a+feHjveNsPLyDArU90uE/PsmrkT4O8sE+PUjwPugbDj8yqCI/bpQ1PzawRj/Fz1U/bcxiP/eEbT/13XU/D8J7PzIifz+79X8/jzp+Pxr1eT9JMHM/bf1pPw10Xj+ssVA/etlAPwEULz+zjhs/gHsGP6Ig4D75DLE+cDSAPvQnHD5lXlk9BQ9AvUvkFb5uRXq+NBOuvm1G3b7VIQW/6ksav1PrLb/gzT+/0cVPv02qXb/MV2m/b7Byv02ceb+zCX6/Te1/v0ZCf79VCny/tE12vxAbbr9gh2O/tK1Wv+quR79csTa/gOAjv3dsD78lE/O+m9/Evn20lL4DGka+qKDBvcIfrjvKSNc9d8RQPkTnmT6j4sk+nNn3Pk+rET829SU/nZY4P91fST8GJlg/TsNkP3EXbz8DCHc/toB8P4tzfz/22H8/9a99Pw7+eD9Dz3E/9DVoP65KXD/uK04/0f09P7zpKz/sHRg/A80CPxBb2D6x8qg+WrRvPjAeCz64kBQ9qmmCvQ7nJr4vd4W+cyW2vs8B5b4zygi/m7Udv5wNMb+9oEK/CEJSv4TJX7+VFGu/VwZ0v+uHer+siH6/XP5/v0Dlfr8nQHu/ZBh1v7R9bL8bhmG/pk1Uvyb2RL/YpjO/BIwgv4jWC7+3duu+Fua8vkdyjL5zLjW+0FGfvSxDtTx8f/k9/5dhPpsXoj6bxNE+C1n/PiswFT82Nik/b4s7P/EATD+ibFo/oKlmP52YcD8wIHg/Fy19P2eyfz+tqX8//xJ9P/30dz+9XHA/rF1mP10RWj9Dl0s/aBQ7PwazKD8hohQ/Gyr+PtWF0D4vzKA+e+5ePrcU9D2VcJ88YcKkvb3dN779wY2+gyq+vp6s7L6raAy/4hMhvxUjNL+DZUW/CK9Uv4jYYb9ZwGy/l0p1v2dhe7859X6/5Px/v8d1fr/KY3q/VtFzvzvPar+EdF+/Ot5RvyAvQr9UjzC/6isdv3o2CL8/yeO+5d60vuYlhL7FNSS+3u55vaR4Hz0Q0g0+NFtyPjc8qj5kl9k+/2IDPzuqGD/3aiw/rnI+P0GTTj9vo1w/Q

For high-dimensional arrays, the individual axis labels indicate which level of the plot corresponds to which axis. Above, each 7x8 square facet represents a slice `my_4d_array[i,j,:,:]`, with individual pixels ranging along axis 2 and axis 3; this is denoted by the `axis2` and `axis3` labels for that facet. The six columns correspond to slices along axis 1, and the five rows correspond to slices along axis 0, as denoted by the outermost labels for those axes.

You can control which axes get assigned to which direction if you want, specified from innermost to outermost:

In [12]:
pz.ts.render_array(my_4d_array, columns=[2, 0, 1])

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACAP5i4fj+l5Xo/75B0P6fKaz9AqWA/MklTP7PMQz9fWzI/1SEfP0BRCj/APeg+9Ya5Ppv1iD7eCy4+qt6QPdYz77zQ7wO+qqdovjaGpb4zEdW+mD0BvwKoFr8RkSq/1sU8v78XTb8QXVu/RnFnv3w1cb+3kHi/JnB9v1LHf78/kH+/ect8vxaAd7+gu2+/9pFlvxUdWb/cfEq/s9Y5vzBVJ7+qJxO/dwP7vo01zb68Wp2+29pXvjmw5b3S+Uq8qzKzPRn9Pj4sPJE+TIbBPmvh7z566w0/O3siP2JrNT98i0Y/va9VP2ixYj86b20/uM11P3u3ez9iHX8/vPZ/P15Bfj+mAXo/cUJzPwQVaj/VkF4/XNNQP73/QD9zPi8/6bwbPwStBj9GieA+L3qxPiGlgD4KDh0+XABdPbpsPL32/RS+o2N5vrWlrb5t3dy+GvAEv3QdGr+ZwC2/Tqc/v8ujT78pjV2/1T9pv+Kdcr9aj3m/ewJ+v+Lrf7+tRn+/gRR8v4tddr9rMG6/CKJjv2TNVr9R00e/Hdo2vzMNJL+qnA+/lHnzvgxLxb7eI5W+a/5GvjRww71yBZE7s3nVPYLgTz45eJk+o3fJPrlz9z5uexE/4MglP0NuOD/nO0k/0AZYPyipZD+dAm8/uPh2Pxl3fD+1b38/8tp/P723fT+OC3k/WeJxP29OaD9PaFw/aU5OP88kPj/ZFCw/uEwYPwj/Aj+MxNg+kWCpPq+WcD7XBAw+ZDMYPfKYgL1UASa+yAaFvqK4tb6rmeS+/ZgIv76HHb+O4zC/6HpCv88gUr86rV+/g/1qv7n0c7/te3q/bYJ+v+z9f7+g6n6/Skt7vywpdb/4k2y/oaFhvyduVL9PG0W/S9Azv1e5IL9GBwy/EN7rvkRSvb424oy+lRM2vgkiob0C/a08R7H3Pd+0YD4qqaE+ZFrRPh70/j7aABU/hAopP8tjOz/B3Us/QU5aP1uQZj+0hHA/1xF4P3IkfT+Mr38/o6x/P78bfT9wA3g/v3BwPwl3Zj/UL1o/iLpLPx88Oz/J3ig/gtEUPyOP/j4i8NA+sTqhPrLRXz4F4/U90LamPDnyor2t+Da+HVKNvmm+vb5dRey+/DcMv6DmIL+1+TO/bkBFv52OVL8avWG/L6psv+g5db9fVnu/8+9+v2/9f78hfH6/4m96vw7jc79l5mq/5ZBfv4r/Ub8JVUK/dbkwv9hZHb+/Zwi/ezHkvshLtb5aloS+kRslvmiQfb0I1hs9d+sMPvh4cT5pzqk+/i3ZPgcxAz9+exg/7D8sP8RLPj/acE4/5YVcP91maD9i9XE/ARl5P3i/fT/g3H8/0mt/P29tfD9f6XY/ve1uP/aOZD+P51c/5xdJP+BFOD+BnCU/hUsRP8gN9z6YDMk+JQmZPoL8Tj6QqtM9NtZnO7U/xb3J4ke+OJOVvnO2xb723/O+1swPv945JL/VAje/rvdHvwntVr+kvGO/uUVuv1dtdr+gHny/Bkt/v2rqf782+32/WoJ5v0mLcr/SJ2m/+m9dv7qBT7+zgD+/1ZUtv/buGb9YvgS/YXTcvi04rb7NgXi+mRcUvmTKOL1IomA9GPQdPswVgT5d57E+4PHgPoHeBj8Y6xs/3WgvP/YlQT8C9VA/kq1eP44saj+NVHM/JQ56PyBIfj+w938/hRh/P9qsez9uvXU/cVltP1eWYj+qj1U/uWZGP0xCNT88TiI/BbsNP4x67z6BGsE+hsyQPlcYPj7JYrE9t4ZZvOp+571tvli+f8mdvi6gzb7naPu+R1cTvzmBJ7+3/jm/dKBKv+c7Wb+yq2W/BdBvv++Od7+h1Hy/npN/v+DEf7/pZ32/w4J4v/Yhcb9fWGe/CT9bv+T0TL+Anjy/pWUqv+94Fr9WCwG/U6fUvggYpb7oxGe+6ggDvgLu57wjr5I9SfEuPsNliT5w87k+eqXoPjiCCj9pTx8/HIUyPy3yQz8KalM/IsVgP0zhaz8conQ/L/F6P2G+fj/5/38/wrJ+Pw7aej+1f3Q/97NrP1ONYD9PKFM/LqdDP5kxMj849B4/QiAKP/vV5z5wGrk+bIWIPmkmLT4rDo89nnn2vK/WBL5himm+W/Slvgh71b7TbwG/DtcWv3S8Kr8i7Ty/kDpNvwt7W78gime/9Uhxv52eeL9WeH2/t8l/v9OMf79Ewny/MHF3vy+nb78ueGW/Of5YvzpZSr+mrjm/HyknvwX4Er/7nfq+4MrMvvHrnL4991a+feHjveNsPLyDArU90uE/PsmrkT4O8sE+PUjwPugbDj8yqCI/bpQ1PzawRj/Fz1U/bcxiP/eEbT/13XU/D8J7PzIifz+79X8/jzp+Pxr1eT9JMHM/bf1pPw10Xj+ssVA/etlAPwEULz+zjhs/gHsGP6Ig4D75DLE+cDSAPvQnHD5lXlk9BQ9AvUvkFb5uRXq+NBOuvm1G3b7VIQW/6ksav1PrLb/gzT+/0cVPv02qXb/MV2m/b7Byv02ceb+zCX6/Te1/v0ZCf79VCny/tE12vxAbbr9gh2O/tK1Wv+quR79csTa/gOAjv3dsD78lE/O+m9/Evn20lL4DGka+qKDBvcIfrjvKSNc9d8RQPkTnmT6j4sk+nNn3Pk+rET829SU/nZY4P91fST8GJlg/TsNkP3EXbz8DCHc/toB8P4tzfz/22H8/9a99Pw7+eD9Dz3E/9DVoP65KXD/uK04/0f09P7zpKz/sHRg/A80CPxBb2D6x8qg+WrRvPjAeCz64kBQ9qmmCvQ7nJr4vd4W+cyW2vs8B5b4zygi/m7Udv5wNMb+9oEK/CEJSv4TJX7+VFGu/VwZ0v+uHer+siH6/XP5/v0Dlfr8nQHu/ZBh1v7R9bL8bhmG/pk1Uvyb2RL/YpjO/BIwgv4jWC7+3duu+Fua8vkdyjL5zLjW+0FGfvSxDtTx8f/k9/5dhPpsXoj6bxNE+C1n/PiswFT82Nik/b4s7P/EATD+ibFo/oKlmP52YcD8wIHg/Fy19P2eyfz+tqX8//xJ9P/30dz+9XHA/rF1mP10RWj9Dl0s/aBQ7PwazKD8hohQ/Gyr+PtWF0D4vzKA+e+5ePrcU9D2VcJ88YcKkvb3dN779wY2+gyq+vp6s7L6raAy/4hMhvxUjNL+DZUW/CK9Uv4jYYb9ZwGy/l0p1v2dhe7859X6/5Px/v8d1fr/KY3q/VtFzvzvPar+EdF+/Ot5RvyAvQr9UjzC/6isdv3o2CL8/yeO+5d60vuYlhL7FNSS+3u55vaR4Hz0Q0g0+NFtyPjc8qj5kl9k+/2IDPzuqGD/3aiw/rnI+P0GTTj9vo1w/Q

Note that the gap between the "axis0" groups is twice as large as the gap between "axis2" groups, so that you can visually distinguish the groups.

Arrayviz can also visualize `NamedArray`, and takes labels from them. This means that, if your code is written with `NamedArrays`, you get labeled visualizations for free! This applies both to axes that have been tagged with a name and axes that haven't. (See the [NamedArray tutorial](named_axes.ipynb) for more information on how `NamedArray`s work in penzai.)

In [13]:
col = pz.nx.wrap(np.linspace(-2, 2, 31)).tag("col")
row = pz.nx.wrap(np.linspace(-2, 2, 31)).tag("row")
sign = pz.nx.wrap(np.array([1, -1])).tag("sign")

my_named_array = sign * (col**2 + row**2)

pz.ts.render_array(my_named_array, columns=["col", "sign"])

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AAAAQZKA70BpJOBAhuvRQOfVxECO47hAexSuQK1opEAk4JtA4XqUQOQ4jkArGolAuB6FQItGgkCjkYBAAACAQKORgECLRoJAuB6FQCsaiUDkOI5A4XqUQCTgm0CtaKRAexSuQI7juEDn1cRAhuvRQGkk4ECSgO9AAAAAQZKA70AjAd9A+qTPQBdswUB4VrRAIGSoQAyVnUA/6ZNAtmCLQHP7g0DqcntAejVxQJQ+aUA5jmNAaCRgQCMBX0BoJGBAOY5jQJQ+aUB6NXFA6nJ7QHP7g0C2YItAP+mTQAyVnUAgZKhAeFa0QBdswUD6pM9AIwHfQJKA70BpJOBA+qTPQNJIwEDuD7JAUPqkQPgHmUDkOI5AFo2EQBsJeECVPmlAmbpcQCl9UkBDhkpA6NVEQBdsQUDSSEBAF2xBQOjVREBDhkpAKX1SQJm6XECVPmlAGwl4QBaNhEDkOI5A+AeZQFD6pEDuD7JA0kjAQPqkz0BpJOBAhuvRQBdswUDuD7JAC9ejQGzBlkAUz4pAAACAQGaobEBUl1tAzsxMQNJIQEBiCzZAfBQuQCFkKEBQ+iRAC9cjQFD6JEAhZChAfBQuQGILNkDSSEBAzsxMQFSXW0BmqGxAAACAQBTPikBswZZAC9ejQO4PskAXbMFAhuvRQOfVxEB4VrRAUPqkQGzBlkDOq4lA63J7QMTUZUAofVJAF2xBQJGhMkCVHSZAJeAbQD/pE0DkOA5AE88KQM6rCUATzwpA5DgOQD/pE0Al4BtAlR0mQJGhMkAXbEFAKH1SQMTUZUDrcntAzquJQGzBlkBQ+qRAeFa0QOfVxECO47hAIGSoQPgHmUAUz4pA63J7QDqOY0AT8E1AeJg6QGaHKUDgvBpA5DgOQHT7A0AbCfg/ZajsP8XU5T86juM/xdTlP2Wo7D8bCfg/dPsDQOQ4DkDgvBpAZocpQHiYOkAT8E1AOo5jQOtye0AUz4pA+AeZQCBkqECO47hAexSuQAyVnUDkOI5AAACAQMTUZUAT8E1A7FE4QFD6JEA/6RNAuR4FQHo18T+Zutw/zczMPxdswT93mLo/7FG4P3eYuj8XbME/zczMP5m63D96NfE/uR4FQD/pE0BQ+iRA7FE4QBPwTUDE1GVAAACAQOQ4jkAMlZ1AexSuQK1opEA/6ZNAFo2EQGaobEAofVJAeJg6QFD6JEC1ohFAo5EAQDqO4z9Ehso/Ygu2P5Ydpj/gvJo/QOmTP7WikT9A6ZM/4LyaP5Ydpj9iC7Y/RIbKPzqO4z+jkQBAtaIRQFD6JEB4mDpAKH1SQGaobEAWjYRAP+mTQK1opEAk4JtAtmCLQBsJeEBUl1tAF2xBQGaHKUA/6RNAo5EAQCMB3z8XbME/IGSoPz/pkz9z+4M/ejVxPzmOYz8jAV8/OY5jP3o1cT9z+4M/P+mTPyBkqD8XbME/IwHfP6ORAEA/6RNAZocpQBdsQUBUl1tAGwl4QLZgi0Ak4JtA4XqUQHP7g0CVPmlAzsxMQJGhMkDgvBpAuR4FQDqO4z8XbME/C9ejPxTPij9mqGw/zsxMP2ILNj8hZCg/C9cjPyFkKD9iCzY/zsxMP2aobD8Uz4o/C9ejPxdswT86juM/uR4FQOC8GkCRoTJAzsxMQJU+aUBz+4NA4XqUQOQ4jkDqcntAmbpcQNJIQECVHSZA5DgOQHo18T9Ehso/IGSoPxTPij86jmM/eJg6P+C8Gj90+wM/ZajsPjqO4z5lqOw+dPsDP+C8Gj94mDo/Oo5jPxTPij8gZKg/RIbKP3o18T/kOA5AlR0mQNJIQECZulxA6nJ7QOQ4jkArGolAejVxQCl9UkBiCzZAJeAbQHT7A0CZutw/Ygu2Pz/pkz9mqGw/eJg6P7WiET86juM+Ygu2PuC8mj61opE+4LyaPmILtj46juM+taIRP3iYOj9mqGw/P+mTP2ILtj+Zutw/dPsDQCXgG0BiCzZAKX1SQHo1cUArGolAuB6FQJQ+aUBDhkpAfBQuQD/pE0AbCfg/zczMP5Ydpj9z+4M/zsxMP+C8Gj86juM+C9ejPmaobD5iCzY+C9cjPmILNj5mqGw+C9ejPjqO4z7gvBo/zsxMP3P7gz+WHaY/zczMPxsJ+D8/6RNAfBQuQEOGSkCUPmlAuB6FQItGgkA5jmNA6NVEQCFkKEDkOA5AZajsPxdswT/gvJo/ejVxP2ILNj90+wM/Ygu2PmaobD61ohE+Ygu2PbWikT1iC7Y9taIRPmaobD5iC7Y+dPsDP2ILNj96NXE/4LyaPxdswT9lqOw/5DgOQCFkKEDo1URAOY5jQItGgkCjkYBAaCRgQBdsQUBQ+iRAE88KQMXU5T93mLo/QOmTPzmOYz8hZCg/ZajsPuC8mj5iCzY+Ygu2PbWiET21opE8taIRPWILtj1iCzY+4LyaPmWo7D4hZCg/OY5jP0Dpkz93mLo/xdTlPxPPCkBQ+iRAF2xBQGgkYECjkYBAAACAQCMBX0DSSEBAC9cjQM6rCUA6juM/7FG4P7WikT8jAV8/C9cjPzqO4z61opE+C9cjPrWikT21opE8AAAAALWikTy1opE9C9cjPrWikT46juM+C9cjPyMBXz+1opE/7FG4PzqO4z/OqwlAC9cjQNJIQEAjAV9AAACAQKORgEBoJGBAF2xBQFD6JEATzwpAxdTlP3eYuj9A6ZM/OY5jPyFkKD9lqOw+4LyaPmILNj5iC7Y9taIRPbWikTy1ohE9Ygu2PWILNj7gvJo+ZajsPiFkKD85jmM/QOmTP3eYuj/F1OU/E88KQFD6JEAXbEFAaCRgQKORgECLRoJAOY5jQOjVREAhZChA5DgOQGWo7D8XbME/4LyaP3o1cT9iCzY/dPsDP2ILtj5mqGw+taIRPmILtj21opE9Ygu2PbWiET5mqGw+Ygu2PnT7Az9iCzY/ejVxP+C8mj8XbME/ZajsP+Q4DkAhZChA6NVEQDmOY0CLRoJAuB6FQJQ+aUBDhkpAfBQuQD/pE0AbCfg/zczMP5Ydpj9z+4M/zsxMP+C8Gj86juM+C9ejPmaobD5iCzY+C9cjPmILNj5mqGw+C9ejPjqO4z7gvBo/zsxMP3P7gz+WHaY/zczMPxsJ+D8/6RNAfBQuQEOGSkCUPmlAuB6FQCsaiUB6NXFAKX1SQGILNkAl4BtAdPsDQJm63D9iC7Y/P+mTP2aobD94mDo/taIRPzqO4z5iC7Y+4LyaPrWikT7gvJo+Ygu2PjqO4z61ohE/eJg6P2aobD8/6ZM/Ygu2P5m63D90+wNAJeAbQGILNkApfVJAejVxQCsaiUDkOI5A6nJ7QJm6XEDSSEBAl

In [14]:
pz.ts.render_array(my_named_array.untag("sign"))

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AAAAQZKA70BpJOBAhuvRQOfVxECO47hAexSuQK1opEAk4JtA4XqUQOQ4jkArGolAuB6FQItGgkCjkYBAAACAQKORgECLRoJAuB6FQCsaiUDkOI5A4XqUQCTgm0CtaKRAexSuQI7juEDn1cRAhuvRQGkk4ECSgO9AAAAAQZKA70AjAd9A+qTPQBdswUB4VrRAIGSoQAyVnUA/6ZNAtmCLQHP7g0DqcntAejVxQJQ+aUA5jmNAaCRgQCMBX0BoJGBAOY5jQJQ+aUB6NXFA6nJ7QHP7g0C2YItAP+mTQAyVnUAgZKhAeFa0QBdswUD6pM9AIwHfQJKA70BpJOBA+qTPQNJIwEDuD7JAUPqkQPgHmUDkOI5AFo2EQBsJeECVPmlAmbpcQCl9UkBDhkpA6NVEQBdsQUDSSEBAF2xBQOjVREBDhkpAKX1SQJm6XECVPmlAGwl4QBaNhEDkOI5A+AeZQFD6pEDuD7JA0kjAQPqkz0BpJOBAhuvRQBdswUDuD7JAC9ejQGzBlkAUz4pAAACAQGaobEBUl1tAzsxMQNJIQEBiCzZAfBQuQCFkKEBQ+iRAC9cjQFD6JEAhZChAfBQuQGILNkDSSEBAzsxMQFSXW0BmqGxAAACAQBTPikBswZZAC9ejQO4PskAXbMFAhuvRQOfVxEB4VrRAUPqkQGzBlkDOq4lA63J7QMTUZUAofVJAF2xBQJGhMkCVHSZAJeAbQD/pE0DkOA5AE88KQM6rCUATzwpA5DgOQD/pE0Al4BtAlR0mQJGhMkAXbEFAKH1SQMTUZUDrcntAzquJQGzBlkBQ+qRAeFa0QOfVxECO47hAIGSoQPgHmUAUz4pA63J7QDqOY0AT8E1AeJg6QGaHKUDgvBpA5DgOQHT7A0AbCfg/ZajsP8XU5T86juM/xdTlP2Wo7D8bCfg/dPsDQOQ4DkDgvBpAZocpQHiYOkAT8E1AOo5jQOtye0AUz4pA+AeZQCBkqECO47hAexSuQAyVnUDkOI5AAACAQMTUZUAT8E1A7FE4QFD6JEA/6RNAuR4FQHo18T+Zutw/zczMPxdswT93mLo/7FG4P3eYuj8XbME/zczMP5m63D96NfE/uR4FQD/pE0BQ+iRA7FE4QBPwTUDE1GVAAACAQOQ4jkAMlZ1AexSuQK1opEA/6ZNAFo2EQGaobEAofVJAeJg6QFD6JEC1ohFAo5EAQDqO4z9Ehso/Ygu2P5Ydpj/gvJo/QOmTP7WikT9A6ZM/4LyaP5Ydpj9iC7Y/RIbKPzqO4z+jkQBAtaIRQFD6JEB4mDpAKH1SQGaobEAWjYRAP+mTQK1opEAk4JtAtmCLQBsJeEBUl1tAF2xBQGaHKUA/6RNAo5EAQCMB3z8XbME/IGSoPz/pkz9z+4M/ejVxPzmOYz8jAV8/OY5jP3o1cT9z+4M/P+mTPyBkqD8XbME/IwHfP6ORAEA/6RNAZocpQBdsQUBUl1tAGwl4QLZgi0Ak4JtA4XqUQHP7g0CVPmlAzsxMQJGhMkDgvBpAuR4FQDqO4z8XbME/C9ejPxTPij9mqGw/zsxMP2ILNj8hZCg/C9cjPyFkKD9iCzY/zsxMP2aobD8Uz4o/C9ejPxdswT86juM/uR4FQOC8GkCRoTJAzsxMQJU+aUBz+4NA4XqUQOQ4jkDqcntAmbpcQNJIQECVHSZA5DgOQHo18T9Ehso/IGSoPxTPij86jmM/eJg6P+C8Gj90+wM/ZajsPjqO4z5lqOw+dPsDP+C8Gj94mDo/Oo5jPxTPij8gZKg/RIbKP3o18T/kOA5AlR0mQNJIQECZulxA6nJ7QOQ4jkArGolAejVxQCl9UkBiCzZAJeAbQHT7A0CZutw/Ygu2Pz/pkz9mqGw/eJg6P7WiET86juM+Ygu2PuC8mj61opE+4LyaPmILtj46juM+taIRP3iYOj9mqGw/P+mTP2ILtj+Zutw/dPsDQCXgG0BiCzZAKX1SQHo1cUArGolAuB6FQJQ+aUBDhkpAfBQuQD/pE0AbCfg/zczMP5Ydpj9z+4M/zsxMP+C8Gj86juM+C9ejPmaobD5iCzY+C9cjPmILNj5mqGw+C9ejPjqO4z7gvBo/zsxMP3P7gz+WHaY/zczMPxsJ+D8/6RNAfBQuQEOGSkCUPmlAuB6FQItGgkA5jmNA6NVEQCFkKEDkOA5AZajsPxdswT/gvJo/ejVxP2ILNj90+wM/Ygu2PmaobD61ohE+Ygu2PbWikT1iC7Y9taIRPmaobD5iC7Y+dPsDP2ILNj96NXE/4LyaPxdswT9lqOw/5DgOQCFkKEDo1URAOY5jQItGgkCjkYBAaCRgQBdsQUBQ+iRAE88KQMXU5T93mLo/QOmTPzmOYz8hZCg/ZajsPuC8mj5iCzY+Ygu2PbWiET21opE8taIRPWILtj1iCzY+4LyaPmWo7D4hZCg/OY5jP0Dpkz93mLo/xdTlPxPPCkBQ+iRAF2xBQGgkYECjkYBAAACAQCMBX0DSSEBAC9cjQM6rCUA6juM/7FG4P7WikT8jAV8/C9cjPzqO4z61opE+C9cjPrWikT21opE8AAAAALWikTy1opE9C9cjPrWikT46juM+C9cjPyMBXz+1opE/7FG4PzqO4z/OqwlAC9cjQNJIQEAjAV9AAACAQKORgEBoJGBAF2xBQFD6JEATzwpAxdTlP3eYuj9A6ZM/OY5jPyFkKD9lqOw+4LyaPmILNj5iC7Y9taIRPbWikTy1ohE9Ygu2PWILNj7gvJo+ZajsPiFkKD85jmM/QOmTP3eYuj/F1OU/E88KQFD6JEAXbEFAaCRgQKORgECLRoJAOY5jQOjVREAhZChA5DgOQGWo7D8XbME/4LyaP3o1cT9iCzY/dPsDP2ILtj5mqGw+taIRPmILtj21opE9Ygu2PbWiET5mqGw+Ygu2PnT7Az9iCzY/ejVxP+C8mj8XbME/ZajsP+Q4DkAhZChA6NVEQDmOY0CLRoJAuB6FQJQ+aUBDhkpAfBQuQD/pE0AbCfg/zczMP5Ydpj9z+4M/zsxMP+C8Gj86juM+C9ejPmaobD5iCzY+C9cjPmILNj5mqGw+C9ejPjqO4z7gvBo/zsxMP3P7gz+WHaY/zczMPxsJ+D8/6RNAfBQuQEOGSkCUPmlAuB6FQCsaiUB6NXFAKX1SQGILNkAl4BtAdPsDQJm63D9iC7Y/P+mTP2aobD94mDo/taIRPzqO4z5iC7Y+4LyaPrWikT7gvJo+Ygu2PjqO4z61ohE/eJg6P2aobD8/6ZM/Ygu2P5m63D90+wNAJeAbQGILNkApfVJAejVxQCsaiUDkOI5A6nJ7QJm6XEDSSEBAl

### Identifying extreme or invalid array values

By default, arrayviz tries to configure the colormap to show interesting detail, clipping outliers. Specifically, it limits the colormap to 3 standard deviations away from the mean (or, technically, from zero if `around_zero` is set):

In [15]:
my_outlier_array = np.cos(np.arange(300).reshape((10,30)) * 0.2)
my_outlier_array[4, 2] = 10.0
pz.ts.render_array(my_outlier_array)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACAP6Xlej+nyms/MklTP19bMj9AUQo/9Ya5Pt4LLj7WM++8qqdovjMR1b4CqBa/1sU8vxBdW798NXG/JnB9vz+Qf78WgHe/9pFlv9x8Sr8wVSe/dwP7vrxanb45sOW9qzKzPSw8kT5r4e8+O3siP3yLRj9osWI/uM11P2Idfz9eQX4/cUJzP9WQXj+9/0A/6bwbP0aJ4D4hpYA+XABdPfb9FL61pa2+GvAEv5nALb/Lo0+/1T9pv1qPeb/i63+/gRR8v2swbr9kzVa/Hdo2v6qcD78MS8W+a/5GvnIFkTuC4E8+o3fJPm57ET9Dbjg/0AZYP50Cbz8Zd3w/8tp/P44LeT9vTmg/aU5OP9kULD8I/wI/kWCpPtcEDD7ymIC9yAaFvquZ5L6+hx2/6HpCvzqtX7+59HO/bYJ+v6Dqfr8sKXW/oaFhv08bRb9XuSC/EN7rvjbijL4JIqG9R7H3PSqpoT4e9P4+hAopP8HdSz9bkGY/1xF4P4yvfz+/G30/v3BwP9QvWj8fPDs/gtEUPyLw0D6y0V8+0LamPK34Nr5pvr2+/DcMv7X5M7+djlS/L6psv19We79v/X+/4m96v2Xmar+K/1G/dbkwv79nCL/IS7W+kRslvgjWGz34eHE+/i3ZPn57GD8AACBB5YVcP2L1cT94v30/0mt/P1/pdj/2jmQ/5xdJP4GcJT/IDfc+JQmZPpCq0z21P8W9OJOVvvbf877eOSS/rvdHv6S8Y79XbXa/Bkt/vzb7fb9Ji3K/+m9dv7OAP7/27hm/YXTcvs2BeL5kyji9GPQdPl3nsT6B3gY/3WgvPwL1UD+OLGo/JQ56P7D3fz/arHs/cVltP6qPVT9MQjU/BbsNP4EawT5XGD4+t4ZZvG2+WL4uoM2+R1cTv7f+Ob/nO1m/BdBvv6HUfL/gxH+/w4J4v19YZ7/k9Ey/pWUqv1YLAb8IGKW+6ggDviOvkj3DZYk+eqXoPmlPHz8t8kM/IsVgPxyidD9hvn4/wrJ+P7V/dD9TjWA/LqdDPzj0Hj/71ec+bIWIPisOjz2v1gS+W/SlvtNvAb90vCq/kDpNvyCKZ7+dnni/t8l/v0TCfL8vp2+/Of5Yv6auOb8F+BK/4MrMvj33Vr7jbDy80uE/Pg7ywT7oGw4/bpQ1P8XPVT/3hG0/D8J7P7v1fz8a9Xk/bf1pP6yxUD8BFC8/gHsGP/kMsT70Jxw+BQ9AvW5Fer5tRt2+6ksav+DNP79Nql2/b7Byv7MJfr9GQn+/tE12v2CHY7/qrke/gOAjvyUT8759tJS+qKDBvcpI1z1E55k+nNn3Pjb1JT/dX0k/TsNkPwMIdz+Lc38/9a99P0PPcT+uSlw/0f09P+wdGD8QW9g+WrRvPriQFD0O5ya+cyW2vjPKCL+cDTG/CEJSv5UUa7/rh3q/XP5/vydAe7+0fWy/pk1Uv9imM7+I1gu/Fua8vnMuNb4sQ7U8/5dhPpvE0T4rMBU/b4s7P6JsWj+dmHA/Fy19P62pfz/99Hc/rF1mP0OXSz8Gsyg/Gyr+Pi/MoD63FPQ9YcKkvf3Bjb6erOy+4hMhv4NlRb+I2GG/l0p1vzn1fr/HdX6/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 30}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 30", "start": 0, "end": 30}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -2.739762544631958, "max": 2.739762544631958, "dynamic": true, "rawMinAbs": 0.004425697959959507, "rawMaxAbs": 10.0, "cmapData": [[96, 14, 34], [134, 14, 41], [167, 36, 36], [186, 72, 46], [198, 107, 77], [208, 139, 115], [218, 171, 155], [228, 203, 196], [241, 236, 235], [202, 212, 216], [161, 190, 200], [117, 170, 190], [75, 148, 186], [38, 123, 186], [12, 94, 190], [41, 66, 162], [37, 47, 111]]}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": ", "}, {"type": "index", "axis": "a1", "skip_start": 30, "skip_end": 30}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

Arrayviz also annotates any invalid array values by drawing annotations on top of the visualization:

In [16]:
numerator = np.linspace(-5, 5, 31)
denominator = np.linspace(-1, 1, 13)
array_with_infs_and_nans = numerator[None, :] / denominator[:, None]
pz.ts.render_array(array_with_infs_and_nans)

<ipython-input-16-2f103da6dddd>:3: RuntimeWarning: divide by zero encountered in divide
  array_with_infs_and_nans = numerator[None, :] / denominator[:, None]
<ipython-input-16-2f103da6dddd>:3: RuntimeWarning: invalid value encountered in divide
  array_with_infs_and_nans = numerator[None, :] / denominator[:, None]


ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACgQFVVlUCrqopAAACAQKuqakBVVVVAAABAQKuqKkBVVRVAAAAAQFVV1T+rqqo/AACAP6uqKj+rqqo+AAAAgKuqqr6rqiq/AACAv6uqqr9VVdW/AAAAwFVVFcCrqirAAABAwFVVVcCrqmrAAACAwKuqisBVVZXAAACgwAAAwEAzM7NAZmamQJqZmUDNzIxAAACAQGZmZkDNzExAMzMzQJqZGUAAAABAzczMP5qZmT/NzEw/zczMPgAAAIDNzMy+zcxMv5qZmb/NzMy/AAAAwJqZGcAzMzPAzcxMwGZmZsAAAIDAzcyMwJqZmcBmZqbAMzOzwAAAwMAAAPBAAADgQAAA0EAAAMBAAACwQAAAoEAAAJBAAACAQAAAYEAAAEBAAAAgQAAAAEAAAMA/AACAPwAAAD8AAACAAAAAvwAAgL8AAMC/AAAAwAAAIMAAAEDAAABgwAAAgMAAAJDAAACgwAAAsMAAAMDAAADQwAAA4MAAAPDAAAAgQVVVFUGrqgpBAAAAQauq6kBVVdVAAADAQKuqqkBVVZVAAACAQFVVVUCrqipAAAAAQKuqqj+rqio/AAAAgKuqKr+rqqq/AAAAwKuqKsBVVVXAAACAwFVVlcCrqqrAAADAwFVV1cCrqurAAAAAwauqCsFVVRXBAAAgwQAAcEEAAGBBAABQQQAAQEEAADBBAAAgQQAAEEEAAABBAADgQAAAwEAAAKBAAACAQAAAQEAAAABAAACAPwAAAIAAAIC/AAAAwAAAQMAAAIDAAACgwAAAwMAAAODAAAAAwQAAEMEAACDBAAAwwQAAQMEAAFDBAABgwQAAcMEAAPBBAADgQQAA0EEAAMBBAACwQQAAoEEAAJBBAACAQQAAYEEAAEBBAAAgQQAAAEEAAMBAAACAQAAAAEAAAACAAAAAwAAAgMAAAMDAAAAAwQAAIMEAAEDBAABgwQAAgMEAAJDBAACgwQAAsMEAAMDBAADQwQAA4MEAAPDBAACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AADA/wAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAA8MEAAODBAADQwQAAwMEAALDBAACgwQAAkMEAAIDBAABgwQAAQMEAACDBAAAAwQAAwMAAAIDAAAAAwAAAAAAAAABAAACAQAAAwEAAAABBAAAgQQAAQEEAAGBBAACAQQAAkEEAAKBBAACwQQAAwEEAANBBAADgQQAA8EEAAHDBAABgwQAAUMEAAEDBAAAwwQAAIMEAABDBAAAAwQAA4MAAAMDAAACgwAAAgMAAAEDAAAAAwAAAgL8AAAAAAACAPwAAAEAAAEBAAACAQAAAoEAAAMBAAADgQAAAAEEAABBBAAAgQQAAMEEAAEBBAABQQQAAYEEAAHBBAAAgwVVVFcGrqgrBAAAAwauq6sBVVdXAAADAwKuqqsBVVZXAAACAwFVVVcCrqirAAAAAwKuqqr+rqiq/AAAAAKuqKj+rqqo/AAAAQKuqKkBVVVVAAACAQFVVlUCrqqpAAADAQFVV1UCrqupAAAAAQauqCkFVVRVBAAAgQQAA8MAAAODAAADQwAAAwMAAALDAAACgwAAAkMAAAIDAAABgwAAAQMAAACDAAAAAwAAAwL8AAIC/AAAAvwAAAAAAAAA/AACAPwAAwD8AAABAAAAgQAAAQEAAAGBAAACAQAAAkEAAAKBAAACwQAAAwEAAANBAAADgQAAA8EAAAMDAMzOzwGZmpsCamZnAzcyMwAAAgMBmZmbAzcxMwDMzM8CamRnAAAAAwM3MzL+amZm/zcxMv83MzL4AAAAAzczMPs3MTD+amZk/zczMPwAAAECamRlAMzMzQM3MTEBmZmZAAACAQM3MjECamZlAZmamQDMzs0AAAMBAAACgwFVVlcCrqorAAACAwKuqasBVVVXAAABAwKuqKsBVVRXAAAAAwFVV1b+rqqq/AACAv6uqKr+rqqq+AAAAAKuqqj6rqio/AACAP6uqqj9VVdU/AAAAQFVVFUCrqipAAABAQFVVVUCrqmpAAACAQKuqikBVVZVAAACgQA==", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ==", "dataStrides": {"a1": 1, "a0": 31}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 31", "start": 0, "end": 31}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 13", "start": 0, "end": 13}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -26.75568389892578, "max": 26.75568389892578, "dynamic": true, "rawMinAbs": 0.3333333432674408, "rawMaxAbs": 30.0, "cmapData": [[96, 14, 34], [134, 14, 41], [167, 36, 36], [186, 72, 46], [198, 107, 77], [208, 139, 115], [218, 171, 155], [228, 203, 196], [241, 236, 235], [202, 212, 216], [161, 190, 200], [117, 170, 190], [

Above, "`I`" (white on a blue background) denotes positive infinity, "`-I`" (white on a red background) denotes negative infinity, and "`X`" (in magenta on a black background) denotes NaN. (You can also see the outlier-clipping behavior clipping a few of the largest finite values here.)

This works in relative mode too:

In [17]:
pz.ts.render_array(array_with_infs_and_nans, around_zero=False)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Linear colormap from -26.7557 to 26.7557. Hover/click for array data.", "arrayBase64": "AACgQFVVlUCrqopAAACAQKuqakBVVVVAAABAQKuqKkBVVRVAAAAAQFVV1T+rqqo/AACAP6uqKj+rqqo+AAAAgKuqqr6rqiq/AACAv6uqqr9VVdW/AAAAwFVVFcCrqirAAABAwFVVVcCrqmrAAACAwKuqisBVVZXAAACgwAAAwEAzM7NAZmamQJqZmUDNzIxAAACAQGZmZkDNzExAMzMzQJqZGUAAAABAzczMP5qZmT/NzEw/zczMPgAAAIDNzMy+zcxMv5qZmb/NzMy/AAAAwJqZGcAzMzPAzcxMwGZmZsAAAIDAzcyMwJqZmcBmZqbAMzOzwAAAwMAAAPBAAADgQAAA0EAAAMBAAACwQAAAoEAAAJBAAACAQAAAYEAAAEBAAAAgQAAAAEAAAMA/AACAPwAAAD8AAACAAAAAvwAAgL8AAMC/AAAAwAAAIMAAAEDAAABgwAAAgMAAAJDAAACgwAAAsMAAAMDAAADQwAAA4MAAAPDAAAAgQVVVFUGrqgpBAAAAQauq6kBVVdVAAADAQKuqqkBVVZVAAACAQFVVVUCrqipAAAAAQKuqqj+rqio/AAAAgKuqKr+rqqq/AAAAwKuqKsBVVVXAAACAwFVVlcCrqqrAAADAwFVV1cCrqurAAAAAwauqCsFVVRXBAAAgwQAAcEEAAGBBAABQQQAAQEEAADBBAAAgQQAAEEEAAABBAADgQAAAwEAAAKBAAACAQAAAQEAAAABAAACAPwAAAIAAAIC/AAAAwAAAQMAAAIDAAACgwAAAwMAAAODAAAAAwQAAEMEAACDBAAAwwQAAQMEAAFDBAABgwQAAcMEAAPBBAADgQQAA0EEAAMBBAACwQQAAoEEAAJBBAACAQQAAYEEAAEBBAAAgQQAAAEEAAMBAAACAQAAAAEAAAACAAAAAwAAAgMAAAMDAAAAAwQAAIMEAAEDBAABgwQAAgMEAAJDBAACgwQAAsMEAAMDBAADQwQAA4MEAAPDBAACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AADA/wAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAA8MEAAODBAADQwQAAwMEAALDBAACgwQAAkMEAAIDBAABgwQAAQMEAACDBAAAAwQAAwMAAAIDAAAAAwAAAAAAAAABAAACAQAAAwEAAAABBAAAgQQAAQEEAAGBBAACAQQAAkEEAAKBBAACwQQAAwEEAANBBAADgQQAA8EEAAHDBAABgwQAAUMEAAEDBAAAwwQAAIMEAABDBAAAAwQAA4MAAAMDAAACgwAAAgMAAAEDAAAAAwAAAgL8AAAAAAACAPwAAAEAAAEBAAACAQAAAoEAAAMBAAADgQAAAAEEAABBBAAAgQQAAMEEAAEBBAABQQQAAYEEAAHBBAAAgwVVVFcGrqgrBAAAAwauq6sBVVdXAAADAwKuqqsBVVZXAAACAwFVVVcCrqirAAAAAwKuqqr+rqiq/AAAAAKuqKj+rqqo/AAAAQKuqKkBVVVVAAACAQFVVlUCrqqpAAADAQFVV1UCrqupAAAAAQauqCkFVVRVBAAAgQQAA8MAAAODAAADQwAAAwMAAALDAAACgwAAAkMAAAIDAAABgwAAAQMAAACDAAAAAwAAAwL8AAIC/AAAAvwAAAAAAAAA/AACAPwAAwD8AAABAAAAgQAAAQEAAAGBAAACAQAAAkEAAAKBAAACwQAAAwEAAANBAAADgQAAA8EAAAMDAMzOzwGZmpsCamZnAzcyMwAAAgMBmZmbAzcxMwDMzM8CamRnAAAAAwM3MzL+amZm/zcxMv83MzL4AAAAAzczMPs3MTD+amZk/zczMPwAAAECamRlAMzMzQM3MTEBmZmZAAACAQM3MjECamZlAZmamQDMzs0AAAMBAAACgwFVVlcCrqorAAACAwKuqasBVVVXAAABAwKuqKsBVVRXAAAAAwFVV1b+rqqq/AACAv6uqKr+rqqq+AAAAAKuqqj6rqio/AACAP6uqqj9VVdU/AAAAQFVVFUCrqipAAABAQFVVVUCrqmpAAACAQKuqikBVVZVAAACgQA==", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ==", "dataStrides": {"a1": 1, "a0": 31}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 31", "start": 0, "end": 31}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 13", "start": 0, "end": 13}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -26.75568389892578, "max": 26.75568389892578, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": [[68, 1, 84], [72, 20, 103], [72, 38, 119], [69, 55, 129], [63, 71, 136], [57, 85, 140], [50, 100, 142], [45, 113, 142], [40, 125, 142], [35, 138, 141], [31, 150, 139], [32, 163, 134], [41, 175, 127], [5

If you want, you can mask out data by providing a "valid mask". Only values where the mask is True will be rendered; masked-out data is shown in gray with black dots.

In [18]:
valid_mask = np.isfinite(array_with_infs_and_nans) & (np.abs(array_with_infs_and_nans) < 10)
pz.ts.render_array(
    array_with_infs_and_nans,
    valid_mask=valid_mask,
)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AACgQFVVlUCrqopAAACAQKuqakBVVVVAAABAQKuqKkBVVRVAAAAAQFVV1T+rqqo/AACAP6uqKj+rqqo+AAAAgKuqqr6rqiq/AACAv6uqqr9VVdW/AAAAwFVVFcCrqirAAABAwFVVVcCrqmrAAACAwKuqisBVVZXAAACgwAAAwEAzM7NAZmamQJqZmUDNzIxAAACAQGZmZkDNzExAMzMzQJqZGUAAAABAzczMP5qZmT/NzEw/zczMPgAAAIDNzMy+zcxMv5qZmb/NzMy/AAAAwJqZGcAzMzPAzcxMwGZmZsAAAIDAzcyMwJqZmcBmZqbAMzOzwAAAwMAAAPBAAADgQAAA0EAAAMBAAACwQAAAoEAAAJBAAACAQAAAYEAAAEBAAAAgQAAAAEAAAMA/AACAPwAAAD8AAACAAAAAvwAAgL8AAMC/AAAAwAAAIMAAAEDAAABgwAAAgMAAAJDAAACgwAAAsMAAAMDAAADQwAAA4MAAAPDAAAAgQVVVFUGrqgpBAAAAQauq6kBVVdVAAADAQKuqqkBVVZVAAACAQFVVVUCrqipAAAAAQKuqqj+rqio/AAAAgKuqKr+rqqq/AAAAwKuqKsBVVVXAAACAwFVVlcCrqqrAAADAwFVV1cCrqurAAAAAwauqCsFVVRXBAAAgwQAAcEEAAGBBAABQQQAAQEEAADBBAAAgQQAAEEEAAABBAADgQAAAwEAAAKBAAACAQAAAQEAAAABAAACAPwAAAIAAAIC/AAAAwAAAQMAAAIDAAACgwAAAwMAAAODAAAAAwQAAEMEAACDBAAAwwQAAQMEAAFDBAABgwQAAcMEAAPBBAADgQQAA0EEAAMBBAACwQQAAoEEAAJBBAACAQQAAYEEAAEBBAAAgQQAAAEEAAMBAAACAQAAAAEAAAACAAAAAwAAAgMAAAMDAAAAAwQAAIMEAAEDBAABgwQAAgMEAAJDBAACgwQAAsMEAAMDBAADQwQAA4MEAAPDBAACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AACA/wAAgP8AAID/AADA/wAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAAgH8AAIB/AACAfwAA8MEAAODBAADQwQAAwMEAALDBAACgwQAAkMEAAIDBAABgwQAAQMEAACDBAAAAwQAAwMAAAIDAAAAAwAAAAAAAAABAAACAQAAAwEAAAABBAAAgQQAAQEEAAGBBAACAQQAAkEEAAKBBAACwQQAAwEEAANBBAADgQQAA8EEAAHDBAABgwQAAUMEAAEDBAAAwwQAAIMEAABDBAAAAwQAA4MAAAMDAAACgwAAAgMAAAEDAAAAAwAAAgL8AAAAAAACAPwAAAEAAAEBAAACAQAAAoEAAAMBAAADgQAAAAEEAABBBAAAgQQAAMEEAAEBBAABQQQAAYEEAAHBBAAAgwVVVFcGrqgrBAAAAwauq6sBVVdXAAADAwKuqqsBVVZXAAACAwFVVVcCrqirAAAAAwKuqqr+rqiq/AAAAAKuqKj+rqqo/AAAAQKuqKkBVVVVAAACAQFVVlUCrqqpAAADAQFVV1UCrqupAAAAAQauqCkFVVRVBAAAgQQAA8MAAAODAAADQwAAAwMAAALDAAACgwAAAkMAAAIDAAABgwAAAQMAAACDAAAAAwAAAwL8AAIC/AAAAvwAAAAAAAAA/AACAPwAAwD8AAABAAAAgQAAAQEAAAGBAAACAQAAAkEAAAKBAAACwQAAAwEAAANBAAADgQAAA8EAAAMDAMzOzwGZmpsCamZnAzcyMwAAAgMBmZmbAzcxMwDMzM8CamRnAAAAAwM3MzL+amZm/zcxMv83MzL4AAAAAzczMPs3MTD+amZk/zczMPwAAAECamRlAMzMzQM3MTEBmZmZAAACAQM3MjECamZlAZmamQDMzs0AAAMBAAACgwFVVlcCrqorAAACAwKuqasBVVVXAAABAwKuqKsBVVRXAAAAAwFVV1b+rqqq/AACAv6uqKr+rqqq+AAAAAKuqqj6rqio/AACAP6uqqj9VVdU/AAAAQFVVFUCrqipAAABAQFVVVUCrqmpAAACAQKuqikBVVZVAAACgQA==", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAAEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAAAAAAAAAAEBAQEBAQEBAQEBAQEBAQEBAQEBAAAAAAAAAAAAAAAAAAAAAQEBAQEBAQEBAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQEBAQEBAQEBAAAAAAAAAAAAAAAAAAAAAAABAQEBAQEBAQEBAQEBAQEBAQEBAQAAAAAAAAEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAAEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ==", "dataStrides": {"a1": 1, "a0": 31}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 31", "start": 0, "end": 31}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 13", "start": 0, "end": 13}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -10.0, "max": 10.0, "dynamic": true, "rawMinAbs": 0.3333333432674408, "rawMaxAbs": 10.0, "cmapData": [[96, 14, 34], [134, 14, 41], [167, 36, 36], [186, 72, 46], [198, 107, 77], [208, 139, 115], [218, 171, 155], [228, 203, 196], [241, 236, 235], [202, 212, 216], [161, 190, 200], [117, 170, 190], [75, 148, 186], [38, 123, 1

### Visualizing categorical data

Arrayviz also supports rendering categorical data, even with very high numbers of categories. Data with a discrete (integer or boolean) dtype is rendered as categorical by default, with different colors for different categories:

In [19]:
pz.ts.render_array(np.arange(10))

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Showing integer digits as nested squares. Hover/click for array data.", "arrayBase64": "AAAAAAEAAAACAAAAAwAAAAQAAAAFAAAABgAAAAcAAAAIAAAACQAAAA==", "arrayDtype": "int32", "validMaskBase64": "AQEBAQEBAQEBAQ==", "dataStrides": {"a0": 1}, "xAxisSpecs": [{"name": "a0", "label": "axis 0: 10", "start": 0, "end": 10}], "yAxisSpecs": [], "slicedAxisSpecs": [], "colormapConfig": {"type": "digitbox", "min": null, "max": null, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": []}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 10, "skip_end": 10}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

In [20]:
pz.ts.render_array(np.array([True, False, False, True, True]))

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Showing integer digits as nested squares. Hover/click for array data.", "arrayBase64": "AQAAAAAAAAAAAAAAAQAAAAEAAAA=", "arrayDtype": "int32", "validMaskBase64": "AQEBAQE=", "dataStrides": {"a0": 1}, "xAxisSpecs": [{"name": "a0", "label": "axis 0: 5", "start": 0, "end": 5}], "yAxisSpecs": [], "slicedAxisSpecs": [], "colormapConfig": {"type": "digitbox", "min": null, "max": null, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": []}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "["}, {"type": "index", "axis": "a0", "skip_start": 5, "skip_end": 5}, {"type": "literal", "value": "]"}, {"type": "literal", "value": "\\n  = "}, {"type": "value"}]}</script></template></div>')

The values from 0 to 9 are rendered with solid colors, with 0 represented as white. Larger numbers are rendered using nested box patterns, with one box per digit of the number, and the color of the box indicating the value of the digit:

In [21]:
pz.ts.render_array(np.arange(1000).reshape((10,100)))

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Showing integer digits as nested squares. Hover/click for array data.", "arrayBase64": "AAAAAAEAAAACAAAAAwAAAAQAAAAFAAAABgAAAAcAAAAIAAAACQAAAAoAAAALAAAADAAAAA0AAAAOAAAADwAAABAAAAARAAAAEgAAABMAAAAUAAAAFQAAABYAAAAXAAAAGAAAABkAAAAaAAAAGwAAABwAAAAdAAAAHgAAAB8AAAAgAAAAIQAAACIAAAAjAAAAJAAAACUAAAAmAAAAJwAAACgAAAApAAAAKgAAACsAAAAsAAAALQAAAC4AAAAvAAAAMAAAADEAAAAyAAAAMwAAADQAAAA1AAAANgAAADcAAAA4AAAAOQAAADoAAAA7AAAAPAAAAD0AAAA+AAAAPwAAAEAAAABBAAAAQgAAAEMAAABEAAAARQAAAEYAAABHAAAASAAAAEkAAABKAAAASwAAAEwAAABNAAAATgAAAE8AAABQAAAAUQAAAFIAAABTAAAAVAAAAFUAAABWAAAAVwAAAFgAAABZAAAAWgAAAFsAAABcAAAAXQAAAF4AAABfAAAAYAAAAGEAAABiAAAAYwAAAGQAAABlAAAAZgAAAGcAAABoAAAAaQAAAGoAAABrAAAAbAAAAG0AAABuAAAAbwAAAHAAAABxAAAAcgAAAHMAAAB0AAAAdQAAAHYAAAB3AAAAeAAAAHkAAAB6AAAAewAAAHwAAAB9AAAAfgAAAH8AAACAAAAAgQAAAIIAAACDAAAAhAAAAIUAAACGAAAAhwAAAIgAAACJAAAAigAAAIsAAACMAAAAjQAAAI4AAACPAAAAkAAAAJEAAACSAAAAkwAAAJQAAACVAAAAlgAAAJcAAACYAAAAmQAAAJoAAACbAAAAnAAAAJ0AAACeAAAAnwAAAKAAAAChAAAAogAAAKMAAACkAAAApQAAAKYAAACnAAAAqAAAAKkAAACqAAAAqwAAAKwAAACtAAAArgAAAK8AAACwAAAAsQAAALIAAACzAAAAtAAAALUAAAC2AAAAtwAAALgAAAC5AAAAugAAALsAAAC8AAAAvQAAAL4AAAC/AAAAwAAAAMEAAADCAAAAwwAAAMQAAADFAAAAxgAAAMcAAADIAAAAyQAAAMoAAADLAAAAzAAAAM0AAADOAAAAzwAAANAAAADRAAAA0gAAANMAAADUAAAA1QAAANYAAADXAAAA2AAAANkAAADaAAAA2wAAANwAAADdAAAA3gAAAN8AAADgAAAA4QAAAOIAAADjAAAA5AAAAOUAAADmAAAA5wAAAOgAAADpAAAA6gAAAOsAAADsAAAA7QAAAO4AAADvAAAA8AAAAPEAAADyAAAA8wAAAPQAAAD1AAAA9gAAAPcAAAD4AAAA+QAAAPoAAAD7AAAA/AAAAP0AAAD+AAAA/wAAAAABAAABAQAAAgEAAAMBAAAEAQAABQEAAAYBAAAHAQAACAEAAAkBAAAKAQAACwEAAAwBAAANAQAADgEAAA8BAAAQAQAAEQEAABIBAAATAQAAFAEAABUBAAAWAQAAFwEAABgBAAAZAQAAGgEAABsBAAAcAQAAHQEAAB4BAAAfAQAAIAEAACEBAAAiAQAAIwEAACQBAAAlAQAAJgEAACcBAAAoAQAAKQEAACoBAAArAQAALAEAAC0BAAAuAQAALwEAADABAAAxAQAAMgEAADMBAAA0AQAANQEAADYBAAA3AQAAOAEAADkBAAA6AQAAOwEAADwBAAA9AQAAPgEAAD8BAABAAQAAQQEAAEIBAABDAQAARAEAAEUBAABGAQAARwEAAEgBAABJAQAASgEAAEsBAABMAQAATQEAAE4BAABPAQAAUAEAAFEBAABSAQAAUwEAAFQBAABVAQAAVgEAAFcBAABYAQAAWQEAAFoBAABbAQAAXAEAAF0BAABeAQAAXwEAAGABAABhAQAAYgEAAGMBAABkAQAAZQEAAGYBAABnAQAAaAEAAGkBAABqAQAAawEAAGwBAABtAQAAbgEAAG8BAABwAQAAcQEAAHIBAABzAQAAdAEAAHUBAAB2AQAAdwEAAHgBAAB5AQAAegEAAHsBAAB8AQAAfQEAAH4BAAB/AQAAgAEAAIEBAACCAQAAgwEAAIQBAACFAQAAhgEAAIcBAACIAQAAiQEAAIoBAACLAQAAjAEAAI0BAACOAQAAjwEAAJABAACRAQAAkgEAAJMBAACUAQAAlQEAAJYBAACXAQAAmAEAAJkBAACaAQAAmwEAAJwBAACdAQAAngEAAJ8BAACgAQAAoQEAAKIBAACjAQAApAEAAKUBAACmAQAApwEAAKgBAACpAQAAqgEAAKsBAACsAQAArQEAAK4BAACvAQAAsAEAALEBAACyAQAAswEAALQBAAC1AQAAtgEAALcBAAC4AQAAuQEAALoBAAC7AQAAvAEAAL0BAAC+AQAAvwEAAMABAADBAQAAwgEAAMMBAADEAQAAxQEAAMYBAADHAQAAyAEAAMkBAADKAQAAywEAAMwBAADNAQAAzgEAAM8BAADQAQAA0QEAANIBAADTAQAA1AEAANUBAADWAQAA1wEAANgBAADZAQAA2gEAANsBAADcAQAA3QEAAN4BAADfAQAA4AEAAOEBAADiAQAA4wEAAOQBAADlAQAA5gEAAOcBAADoAQAA6QEAAOoBAADrAQAA7AEAAO0BAADuAQAA7wEAAPABAADxAQAA8gEAAPMBAAD0AQAA9QEAAPYBAAD3AQAA+AEAAPkBAAD6AQAA+wEAAPwBAAD9AQAA/gEAAP8BAAAAAgAAAQIAAAICAAADAgAABAIAAAUCAAAGAgAABwIAAAgCAAAJAgAACgIAAAsCAAAMAgAADQIAAA4CAAAPAgAAEAIAABECAAASAgAAEwIAABQCAAAVAgAAFgIAABcCAAAYAgAAGQIAABoCAAAbAgAAHAIAAB0CAAAeAgAAHwIAACACAAAhAgAAIgIAACMCAAAkAgAAJQIAACYCAAAnAgAAKAIAACkCAAAqAgAAKwIAACwCAAAtAgAALgIAAC8CAAAwAgAAMQIAADICAAAzAgAANAIAADUCAAA2AgAANwIAADgCAAA5AgAAOgIAADsCAAA8AgAAPQIAAD4CAAA/AgAAQAIAAEECAABCAgAAQwIAAEQCAABFAgAARgIAAEcCAABIAgAASQIAAEoCAABLAgAATAIAAE0CAABOAgAATwIAAFACAABRAgAAUgIAAFMCAABUAgAAVQIAAFYCAABXAgAAWAIAAFkCAABaAgAAWwIAAFwCAABdAgAAXgIAAF8CAABgAgAAYQIAAGICAABjAgAAZAIAAGUCAABmAgAAZwIAAGgCAABpAgAAagIAAGsCAABsAgAAbQIAAG4CAAB

In [22]:
pz.ts.render_array(
    pz.nx.wrap(jnp.arange(20)).tag("a")
    * pz.nx.wrap(jnp.arange(20)).tag("b")
)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Showing integer digits as nested squares. Hover/click for array data.", "arrayBase64": "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAIAAAADAAAABAAAAAUAAAAGAAAABwAAAAgAAAAJAAAACgAAAAsAAAAMAAAADQAAAA4AAAAPAAAAEAAAABEAAAASAAAAEwAAAAAAAAACAAAABAAAAAYAAAAIAAAACgAAAAwAAAAOAAAAEAAAABIAAAAUAAAAFgAAABgAAAAaAAAAHAAAAB4AAAAgAAAAIgAAACQAAAAmAAAAAAAAAAMAAAAGAAAACQAAAAwAAAAPAAAAEgAAABUAAAAYAAAAGwAAAB4AAAAhAAAAJAAAACcAAAAqAAAALQAAADAAAAAzAAAANgAAADkAAAAAAAAABAAAAAgAAAAMAAAAEAAAABQAAAAYAAAAHAAAACAAAAAkAAAAKAAAACwAAAAwAAAANAAAADgAAAA8AAAAQAAAAEQAAABIAAAATAAAAAAAAAAFAAAACgAAAA8AAAAUAAAAGQAAAB4AAAAjAAAAKAAAAC0AAAAyAAAANwAAADwAAABBAAAARgAAAEsAAABQAAAAVQAAAFoAAABfAAAAAAAAAAYAAAAMAAAAEgAAABgAAAAeAAAAJAAAACoAAAAwAAAANgAAADwAAABCAAAASAAAAE4AAABUAAAAWgAAAGAAAABmAAAAbAAAAHIAAAAAAAAABwAAAA4AAAAVAAAAHAAAACMAAAAqAAAAMQAAADgAAAA/AAAARgAAAE0AAABUAAAAWwAAAGIAAABpAAAAcAAAAHcAAAB+AAAAhQAAAAAAAAAIAAAAEAAAABgAAAAgAAAAKAAAADAAAAA4AAAAQAAAAEgAAABQAAAAWAAAAGAAAABoAAAAcAAAAHgAAACAAAAAiAAAAJAAAACYAAAAAAAAAAkAAAASAAAAGwAAACQAAAAtAAAANgAAAD8AAABIAAAAUQAAAFoAAABjAAAAbAAAAHUAAAB+AAAAhwAAAJAAAACZAAAAogAAAKsAAAAAAAAACgAAABQAAAAeAAAAKAAAADIAAAA8AAAARgAAAFAAAABaAAAAZAAAAG4AAAB4AAAAggAAAIwAAACWAAAAoAAAAKoAAAC0AAAAvgAAAAAAAAALAAAAFgAAACEAAAAsAAAANwAAAEIAAABNAAAAWAAAAGMAAABuAAAAeQAAAIQAAACPAAAAmgAAAKUAAACwAAAAuwAAAMYAAADRAAAAAAAAAAwAAAAYAAAAJAAAADAAAAA8AAAASAAAAFQAAABgAAAAbAAAAHgAAACEAAAAkAAAAJwAAACoAAAAtAAAAMAAAADMAAAA2AAAAOQAAAAAAAAADQAAABoAAAAnAAAANAAAAEEAAABOAAAAWwAAAGgAAAB1AAAAggAAAI8AAACcAAAAqQAAALYAAADDAAAA0AAAAN0AAADqAAAA9wAAAAAAAAAOAAAAHAAAACoAAAA4AAAARgAAAFQAAABiAAAAcAAAAH4AAACMAAAAmgAAAKgAAAC2AAAAxAAAANIAAADgAAAA7gAAAPwAAAAKAQAAAAAAAA8AAAAeAAAALQAAADwAAABLAAAAWgAAAGkAAAB4AAAAhwAAAJYAAAClAAAAtAAAAMMAAADSAAAA4QAAAPAAAAD/AAAADgEAAB0BAAAAAAAAEAAAACAAAAAwAAAAQAAAAFAAAABgAAAAcAAAAIAAAACQAAAAoAAAALAAAADAAAAA0AAAAOAAAADwAAAAAAEAABABAAAgAQAAMAEAAAAAAAARAAAAIgAAADMAAABEAAAAVQAAAGYAAAB3AAAAiAAAAJkAAACqAAAAuwAAAMwAAADdAAAA7gAAAP8AAAAQAQAAIQEAADIBAABDAQAAAAAAABIAAAAkAAAANgAAAEgAAABaAAAAbAAAAH4AAACQAAAAogAAALQAAADGAAAA2AAAAOoAAAD8AAAADgEAACABAAAyAQAARAEAAFYBAAAAAAAAEwAAACYAAAA5AAAATAAAAF8AAAByAAAAhQAAAJgAAACrAAAAvgAAANEAAADkAAAA9wAAAAoBAAAdAQAAMAEAAEMBAABWAQAAaQEAAA==", "arrayDtype": "int32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ==", "dataStrides": {"a1": 1, "a0": 20}, "xAxisSpecs": [{"name": "a1", "label": "b: 20", "start": 0, "end": 20}], "yAxisSpecs": [{"name": "a0", "label": "a: 20", "start": 0, "end": 20}], "slicedAxisSpecs": [], "colormapConfig": {"type": "digitbox", "min": null, "max": null, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null, "cmapData": []}, "valueFormattingInstructions": [{"type": "literal", "value": "array"}, {"type": "literal", "value": "[{"}, {"type": "literal", "value": "\'a\':"}, {"type": "index", "axis": "a0", "skip_start": 20, "skip_end": 20}, {"type": "literal", "value": ", \'b\':"}, {"t

You can also render a single integer on its own, if you want (sometimes useful for custom visualizations). Arrayviz supports integers with up to 7 digits.

In [23]:
pz.ts.integer_digitbox(42, size="30px")

ArrayvizRendering(html_src='<span class="inline_digitbox" style="font-size: 30px"><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode;const defns = this.getRootNode().host.defns;defns.runSoon(() => {defns.arrayviz.renderOneDigitbox(parent, {"value": 42, "labelTop": "", "labelBottom": "42"});});</script></treescope-run-here></span>')

In [24]:
pz.ts.integer_digitbox(1234, size="30px")

ArrayvizRendering(html_src='<span class="inline_digitbox" style="font-size: 30px"><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode;const defns = this.getRootNode().host.defns;defns.runSoon(() => {defns.arrayviz.renderOneDigitbox(parent, {"value": 1234, "labelTop": "", "labelBottom": "1234"});});</script></treescope-run-here></span>')

In [25]:
pz.ts.integer_digitbox(7654321, size="30px")

ArrayvizRendering(html_src='<span class="inline_digitbox" style="font-size: 30px"><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode;const defns = this.getRootNode().host.defns;defns.runSoon(() => {defns.arrayviz.renderOneDigitbox(parent, {"value": 7654321, "labelTop": "", "labelBottom": "7654321"});});</script></treescope-run-here></span>')

Negatigve integers render the same way as positive ones, but with a black triangle in the corner indicating the sign:

In [26]:
pz.ts.render_array(np.arange(21 * 21).reshape((21, 21)) - 220)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Showing integer digits as nested squares. Hover/click for array data.", "arrayBase64": "JP///yX///8m////J////yj///8p////Kv///yv///8s////Lf///y7///8v////MP///zH///8y////M////zT///81////Nv///zf///84////Of///zr///87////PP///z3///8+////P////0D///9B////Qv///0P///9E////Rf///0b///9H////SP///0n///9K////S////0z///9N////Tv///0////9Q////Uf///1L///9T////VP///1X///9W////V////1j///9Z////Wv///1v///9c////Xf///17///9f////YP///2H///9i////Y////2T///9l////Zv///2f///9o////af///2r///9r////bP///23///9u////b////3D///9x////cv///3P///90////df///3b///93////eP///3n///96////e////3z///99////fv///3////+A////gf///4L///+D////hP///4X///+G////h////4j///+J////iv///4v///+M////jf///47///+P////kP///5H///+S////k////5T///+V////lv///5f///+Y////mf///5r///+b////nP///53///+e////n////6D///+h////ov///6P///+k////pf///6b///+n////qP///6n///+q////q////6z///+t////rv///6////+w////sf///7L///+z////tP///7X///+2////t////7j///+5////uv///7v///+8////vf///77///+/////wP///8H////C////w////8T////F////xv///8f////I////yf///8r////L////zP///83////O////z////9D////R////0v///9P////U////1f///9b////X////2P///9n////a////2////9z////d////3v///9/////g////4f///+L////j////5P///+X////m////5////+j////p////6v///+v////s////7f///+7////v////8P////H////y////8/////T////1////9v////f////4////+f////r////7/////P////3////+/////////wAAAAABAAAAAgAAAAMAAAAEAAAABQAAAAYAAAAHAAAACAAAAAkAAAAKAAAACwAAAAwAAAANAAAADgAAAA8AAAAQAAAAEQAAABIAAAATAAAAFAAAABUAAAAWAAAAFwAAABgAAAAZAAAAGgAAABsAAAAcAAAAHQAAAB4AAAAfAAAAIAAAACEAAAAiAAAAIwAAACQAAAAlAAAAJgAAACcAAAAoAAAAKQAAACoAAAArAAAALAAAAC0AAAAuAAAALwAAADAAAAAxAAAAMgAAADMAAAA0AAAANQAAADYAAAA3AAAAOAAAADkAAAA6AAAAOwAAADwAAAA9AAAAPgAAAD8AAABAAAAAQQAAAEIAAABDAAAARAAAAEUAAABGAAAARwAAAEgAAABJAAAASgAAAEsAAABMAAAATQAAAE4AAABPAAAAUAAAAFEAAABSAAAAUwAAAFQAAABVAAAAVgAAAFcAAABYAAAAWQAAAFoAAABbAAAAXAAAAF0AAABeAAAAXwAAAGAAAABhAAAAYgAAAGMAAABkAAAAZQAAAGYAAABnAAAAaAAAAGkAAABqAAAAawAAAGwAAABtAAAAbgAAAG8AAABwAAAAcQAAAHIAAABzAAAAdAAAAHUAAAB2AAAAdwAAAHgAAAB5AAAAegAAAHsAAAB8AAAAfQAAAH4AAAB/AAAAgAAAAIEAAACCAAAAgwAAAIQAAACFAAAAhgAAAIcAAACIAAAAiQAAAIoAAACLAAAAjAAAAI0AAACOAAAAjwAAAJAAAACRAAAAkgAAAJMAAACUAAAAlQAAAJYAAACXAAAAmAAAAJkAAACaAAAAmwAAAJwAAACdAAAAngAAAJ8AAACgAAAAoQAAAKIAAACjAAAApAAAAKUAAACmAAAApwAAAKgAAACpAAAAqgAAAKsAAACsAAAArQAAAK4AAACvAAAAsAAAALEAAACyAAAAswAAALQAAAC1AAAAtgAAALcAAAC4AAAAuQAAALoAAAC7AAAAvAAAAL0AAAC+AAAAvwAAAMAAAADBAAAAwgAAAMMAAADEAAAAxQAAAMYAAADHAAAAyAAAAMkAAADKAAAAywAAAMwAAADNAAAAzgAAAM8AAADQAAAA0QAAANIAAADTAAAA1AAAANUAAADWAAAA1wAAANgAAADZAAAA2gAAANsAAADcAAAA", "arrayDtype": "int32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 21}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 21", "start": 0, "end": 21}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 21", "start": 0, "end": 21}], "slicedAxisSpecs": [], "colormapConfig": {"type": "digitbox", "min": null, "max": null, "dynamic": false, "rawMinAbs": null, "rawMaxAbs": null

If your data has a discrete dtype but you don't want to render it as categorical, you can pass the `continuous` flag to render it as numeric instead:

In [27]:
pz.ts.render_array(np.arange(21 * 21).reshape((21, 21)) - 220, continuous=True)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AABcwwAAW8MAAFrDAABZwwAAWMMAAFfDAABWwwAAVcMAAFTDAABTwwAAUsMAAFHDAABQwwAAT8MAAE7DAABNwwAATMMAAEvDAABKwwAAScMAAEjDAABHwwAARsMAAEXDAABEwwAAQ8MAAELDAABBwwAAQMMAAD/DAAA+wwAAPcMAADzDAAA7wwAAOsMAADnDAAA4wwAAN8MAADbDAAA1wwAANMMAADPDAAAywwAAMcMAADDDAAAvwwAALsMAAC3DAAAswwAAK8MAACrDAAApwwAAKMMAACfDAAAmwwAAJcMAACTDAAAjwwAAIsMAACHDAAAgwwAAH8MAAB7DAAAdwwAAHMMAABvDAAAawwAAGcMAABjDAAAXwwAAFsMAABXDAAAUwwAAE8MAABLDAAARwwAAEMMAAA/DAAAOwwAADcMAAAzDAAALwwAACsMAAAnDAAAIwwAAB8MAAAbDAAAFwwAABMMAAAPDAAACwwAAAcMAAADDAAD+wgAA/MIAAPrCAAD4wgAA9sIAAPTCAADywgAA8MIAAO7CAADswgAA6sIAAOjCAADmwgAA5MIAAOLCAADgwgAA3sIAANzCAADawgAA2MIAANbCAADUwgAA0sIAANDCAADOwgAAzMIAAMrCAADIwgAAxsIAAMTCAADCwgAAwMIAAL7CAAC8wgAAusIAALjCAAC2wgAAtMIAALLCAACwwgAArsIAAKzCAACqwgAAqMIAAKbCAACkwgAAosIAAKDCAACewgAAnMIAAJrCAACYwgAAlsIAAJTCAACSwgAAkMIAAI7CAACMwgAAisIAAIjCAACGwgAAhMIAAILCAACAwgAAfMIAAHjCAAB0wgAAcMIAAGzCAABowgAAZMIAAGDCAABcwgAAWMIAAFTCAABQwgAATMIAAEjCAABEwgAAQMIAADzCAAA4wgAANMIAADDCAAAswgAAKMIAACTCAAAgwgAAHMIAABjCAAAUwgAAEMIAAAzCAAAIwgAABMIAAADCAAD4wQAA8MEAAOjBAADgwQAA2MEAANDBAADIwQAAwMEAALjBAACwwQAAqMEAAKDBAACYwQAAkMEAAIjBAACAwQAAcMEAAGDBAABQwQAAQMEAADDBAAAgwQAAEMEAAADBAADgwAAAwMAAAKDAAACAwAAAQMAAAADAAACAvwAAAAAAAIA/AAAAQAAAQEAAAIBAAACgQAAAwEAAAOBAAAAAQQAAEEEAACBBAAAwQQAAQEEAAFBBAABgQQAAcEEAAIBBAACIQQAAkEEAAJhBAACgQQAAqEEAALBBAAC4QQAAwEEAAMhBAADQQQAA2EEAAOBBAADoQQAA8EEAAPhBAAAAQgAABEIAAAhCAAAMQgAAEEIAABRCAAAYQgAAHEIAACBCAAAkQgAAKEIAACxCAAAwQgAANEIAADhCAAA8QgAAQEIAAERCAABIQgAATEIAAFBCAABUQgAAWEIAAFxCAABgQgAAZEIAAGhCAABsQgAAcEIAAHRCAAB4QgAAfEIAAIBCAACCQgAAhEIAAIZCAACIQgAAikIAAIxCAACOQgAAkEIAAJJCAACUQgAAlkIAAJhCAACaQgAAnEIAAJ5CAACgQgAAokIAAKRCAACmQgAAqEIAAKpCAACsQgAArkIAALBCAACyQgAAtEIAALZCAAC4QgAAukIAALxCAAC+QgAAwEIAAMJCAADEQgAAxkIAAMhCAADKQgAAzEIAAM5CAADQQgAA0kIAANRCAADWQgAA2EIAANpCAADcQgAA3kIAAOBCAADiQgAA5EIAAOZCAADoQgAA6kIAAOxCAADuQgAA8EIAAPJCAAD0QgAA9kIAAPhCAAD6QgAA/EIAAP5CAAAAQwAAAUMAAAJDAAADQwAABEMAAAVDAAAGQwAAB0MAAAhDAAAJQwAACkMAAAtDAAAMQwAADUMAAA5DAAAPQwAAEEMAABFDAAASQwAAE0MAABRDAAAVQwAAFkMAABdDAAAYQwAAGUMAABpDAAAbQwAAHEMAAB1DAAAeQwAAH0MAACBDAAAhQwAAIkMAACNDAAAkQwAAJUMAACZDAAAnQwAAKEMAAClDAAAqQwAAK0MAACxDAAAtQwAALkMAAC9DAAAwQwAAMUMAADJDAAAzQwAANEMAADVDAAA2QwAAN0MAADhDAAA5QwAAOkMAADtDAAA8QwAAPUMAAD5DAAA/QwAAQEMAAEFDAABCQwAAQ0MAAERDAABFQwAARkMAAEdDAABIQwAASUMAAEpDAABLQwAATEMAAE1DAABOQwAAT0MAAFBDAABRQwAAUkMAAFNDAABUQwAAVUMAAFZDAABXQwAAWEMAAFlDAABaQwAAW0MAAFxD", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a1": 1, "a0": 21}, "xAxisSpecs": [{"name": "a1", "label": "axis 1: 21", "start": 0, "end": 21}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 21", "start": 0, "end": 21}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -220.0, "max": 220.0, "dynamic": true, "rawMinAbs": 1.0, "rawMaxAbs": 220.0

### Adding axis item labels

For some arrays, it can be useful to associate labels with the individual indices along each axis. For instance, we might want to label a "classes" axis with each individual class, or a "sequence" axis with the tokens of the sequence.

Arrayviz allows you to pass this kind of metadata as an extra argument, and will show it to you when you hover over or click on elements of the array with your mouse.

For positional axes, you can pass any subset of the axes by position:

In [28]:
# Try hovering or clicking:
pz.ts.render_array(
    np.sin(np.linspace(0, 100, 12 * 5 * 7)).reshape((12, 5, 7)),
    axis_item_labels={
        1: ["foo", "bar", "baz", "qux", "xyz"],
        0: ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten", "eleven", "twelve"],
    }
)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AAAAAPITcj5WN+s+iQcoP9bsUD8O+m0/lYl9P5a5fj/VeHE/qIdWPxBtLz8ewfw+81OMPuWRHz3nIUu+IlLZvrNgIL/PAEu/yR5qvwn3e7+Whn+/wZl0vxnPW79pjja/VfQGv2Jnn77icp+96OAjPnYYxz6Gexg/4cVEP4ToZT+PAno/RPB/P5tbdz8awWA/zWg9P6ZTDz/dPLI+2N7uPW3A+L1okbS+FlsQv3g+Pr/kWGG/6Kx3v3j2f79QvXm/wVtlv5L5Q79Cexe/Ec3Evvz2Hr5cXqk9K8ShPooCCD8ebTc/rXFcP/32dD8wmX8/9b17P0KdaT8sPko//mcfP8YQ1z7CQEY+7XQzvQ+4jr5C6v6+eFQwv8g0V7/c4XG/kNh+v8Bcfb/2g22/KzRQv8QWJ7/lAOm+ej1tvgw7nzvz6HY+XGztPkr3KD89pFE/t25uP+K0fT8SmX4/WQ5xPz3ZVT+ZhC4/dJb6Pv7uiT7mrQs90QFQvi2S275vWCG/N8JLv+aear+YLny/cHJ/vwo7dL8xK1u/mq41v07lBb+iCZ2+coaVvdbJKD6oYsk+33oZP/+QRT/kc2Y/SkZ6P4Tofz/PCHc/9SdgP/2RPD9YSw4/PuevPtL75D1AUAG+3OS2vqZhEb/9Ej+/UO9hv7X8d78h+3+/kHZ5v5rNZL8ULEO/E3oWv3qAwr4aDBq+z0izPfofpD7tDwk/t0o4P+wSXT+9UnU/P6p/P1qDez9QGmk/UXpJP1JuHj8eztQ+al5BPt9WR71OG5G+7YgAv886Mb+a4Fe/bElyv//1fr9jLn2/bQxtvz56T7/9JCa/C8nmvpJlaL6ROh88d7x7PvGf7z4F5ik/YFpSP+/hbj+m3n0/BHd+P2eicD+GKVU/FJstP0Vq+D4ziYc+HpLvPHrgVL7l0N2+Mk8iv2SCTL+YHWu/oGR8v75cf7/a2nO/9YVav7HNNL951QS/8KqavhqZi72/sS0+oqvLPkl5Gj/rWkY/4P1mP4KIej83338/hbR2P3WNXz8Iujs/LkINP46QrT5qF9s9gj8GvjU3ub5WZxK/WuY/v2CEYr8DS3i/Pf5/v00ueb8RPmS/aV1Cv/x3Fb+3MsC+SSAVvi0yvT3LeqY+fBwKPzInOT/Vsl0/A611P8O5fz87R3s/95VoPz21SD+xcx0/LIrSPud6PD6cN1u9rH2TvvObAb8UIDK/HotYv4Wvcr/kEX+/fv58v3aTbL8Pv06/NDIlv8yP5L5DjGO+pdZuPDpHgD4U0vE+utMqPz4PUz+1U28/4QZ+P21Tfj8CNXA/hnhUP4KwLD+WPPY+lyKFPv7GxzzZvVm+RQ7gvvpEI79VQU2/35prvyKZfL+CRX+/MHlzv2jfWb+x6zO/1sQDv01LmL7qqoG9m5gyPmDzzT7Fdhs/pSNHP3aGZz82yXo/X9R/P75edj+b8V4/8uA6Pyk4DD/SOKs+rzHRPfQtC75viLu+I2wTv464QL8QGGO/0Jd4v83/f7+J5Hi/J61jv5CNQb/9dBS/y+O9vpIzEL5nGsc9mtSoPjUoCz+PAjo/ZlFeP8sFdj+6x38/lwl7PzUQaD/z7kc/HXgcP/VE0D5Bljc+BxdvvSfflb4wrgK/RQQzv1Q0Wb8nFHO/Pix/vxLNfL8RGWy/oQJOv2w+JL8sVeK+lLFevqQ4nzxzr4I+wQL0PmbAKz/VwlM/CMRvP5Mtfj9MLn4/KMZvPz3GUz/lxCs/aw30Pi27gj6o+p8855levktK4r7FOSS/CP9Nv7gWbL8dzHy/uix/vw4Wc7+JN1m/mggzv2ezAr/A6pW+43dvvWN+Nz7gOdA+UHMcPyrrRz+lDWg/Zgh7P/rHfz95B3Y/aFReP7oGOj9MLQs/DeCoPq9Kxz2OGxC+h9i9vgxwFL+YiUG/Yapjvx3jeL/Q/3+/Q5l4v90aY7+MvEC/GHETv7iTu778RQu+bAHRPWQtqz4WMww/zdw6P6DuXj8YXXY/JtR/P27Kej8MiWc/dCdHP5Z7Gz97/s0+frAyPn96gb25P5i+o78Dv2HnM7853Fm/UXdzvw1Ff78fmny/Pp1rv/RETb+mSSO/LRngvozVWb7/BMc84RaFPvQx9j4IrCw/JHVUP+kycD+8Un4/oQd+P9xVbz+sElM/PtgqP8Xc8T74UoA+tlpwPJ10Y77yhOS+ki0lv3y7Tr8lkWy/kf18v2gSf79zsXK/Wo5Yv28kMr8uoQG/", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a2": 1, "a1": 7, "a0": 35}, "xAxisSpecs": [{"name": "a2", "label": "axis 2: 7", "start": 0, "end": 7}, {"name": "a1", "label": "axis 1: 5", "start": 0, "end": 5}], "yAxisSpecs": [{"name": "a0", "label": "axis 0: 12", "start": 0, "end": 12}], "slicedAxisSpecs": [], "colormapConfig": {"type": "continuous", "min": -0.9999971389770508, "max": 0.9999971389770508, "dynamic": true, "rawMinAbs": 0.004859333857893944, "rawMaxAbs": 0.9999971389770508, "cmapData": [[9

For named axes, you can pass labels by name. Irrelevant labels are simply ignored.

In [29]:
col = pz.nx.wrap(np.linspace(-2, 2, 15)).tag("col")
row = pz.nx.wrap(np.linspace(-2, 2, 15)).tag("row")
sign = pz.nx.wrap(np.array([1, -1])).tag("sign")

my_named_array = sign * (col**2 + row**2)

# Try hovering or clicking:
pz.ts.render_array(
    my_named_array,
    columns=["col", "sign"],
    axis_item_labels={
        "sign": ["positive", "negative"],
        "classes": ["cat", "dog", "mouse", "house"],
    }
)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AAAAQXMK3kBeTsFAwsupQJ2Cl0DwcopAvJyCQAAAgEC8nIJA8HKKQJ2Cl0DCy6lAXk7BQHMK3kAAAABBcwreQOYUvEDRWJ9ANNaHQCAaa0DH+lBAXk5BQOYUPEBeTkFAx/pQQCAaa0A01odA0VifQOYUvEBzCt5AXk7BQNFYn0C8nIJAPzRWQPahMUCdghdANNYHQLycAkA01gdAnYIXQPahMUA/NFZAvJyCQNFYn0BeTsFAwsupQDTWh0A/NFZABi8nQLycAkDI+tA/9qGxPwYvpz/2obE/yPrQP7ycAkAGLydAPzRWQDTWh0DCy6lAnYKXQCAaa0D2oTFAvJwCQOYUvD801oc/x/pQP+YUPD/H+lA/NNaHP+YUvD+8nAJA9qExQCAaa0CdgpdA8HKKQMf6UECdghdAyPrQPzTWhz8GLyc/yPrQPgYvpz7I+tA+Bi8nPzTWhz/I+tA/nYIXQMf6UEDwcopAvJyCQF5OQUA01gdA9qGxP8f6UD/I+tA+Bi8nPgYvpz0GLyc+yPrQPsf6UD/2obE/NNYHQF5OQUC8nIJAAACAQOYUPEC8nAJABi+nP+YUPD8GL6c+Bi+nPQAAAAAGL6c9Bi+nPuYUPD8GL6c/vJwCQOYUPEAAAIBAvJyCQF5OQUA01gdA9qGxP8f6UD/I+tA+Bi8nPgYvpz0GLyc+yPrQPsf6UD/2obE/NNYHQF5OQUC8nIJA8HKKQMf6UECdghdAyPrQPzTWhz8GLyc/yPrQPgYvpz7I+tA+Bi8nPzTWhz/I+tA/nYIXQMf6UEDwcopAnYKXQCAaa0D2oTFAvJwCQOYUvD801oc/x/pQP+YUPD/H+lA/NNaHP+YUvD+8nAJA9qExQCAaa0CdgpdAwsupQDTWh0A/NFZABi8nQLycAkDI+tA/9qGxPwYvpz/2obE/yPrQP7ycAkAGLydAPzRWQDTWh0DCy6lAXk7BQNFYn0C8nIJAPzRWQPahMUCdghdANNYHQLycAkA01gdAnYIXQPahMUA/NFZAvJyCQNFYn0BeTsFAcwreQOYUvEDRWJ9ANNaHQCAaa0DH+lBAXk5BQOYUPEBeTkFAx/pQQCAaa0A01odA0VifQOYUvEBzCt5AAAAAQXMK3kBeTsFAwsupQJ2Cl0DwcopAvJyCQAAAgEC8nIJA8HKKQJ2Cl0DCy6lAXk7BQHMK3kAAAABBAAAAwXMK3sBeTsHAwsupwJ2Cl8DwcorAvJyCwAAAgMC8nILA8HKKwJ2Cl8DCy6nAXk7BwHMK3sAAAADBcwrewOYUvMDRWJ/ANNaHwCAaa8DH+lDAXk5BwOYUPMBeTkHAx/pQwCAaa8A01ofA0VifwOYUvMBzCt7AXk7BwNFYn8C8nILAPzRWwPahMcCdghfANNYHwLycAsA01gfAnYIXwPahMcA/NFbAvJyCwNFYn8BeTsHAwsupwDTWh8A/NFbABi8nwLycAsDI+tC/9qGxvwYvp7/2obG/yPrQv7ycAsAGLyfAPzRWwDTWh8DCy6nAnYKXwCAaa8D2oTHAvJwCwOYUvL801oe/x/pQv+YUPL/H+lC/NNaHv+YUvL+8nALA9qExwCAaa8CdgpfA8HKKwMf6UMCdghfAyPrQvzTWh78GLye/yPrQvgYvp77I+tC+Bi8nvzTWh7/I+tC/nYIXwMf6UMDwcorAvJyCwF5OQcA01gfA9qGxv8f6UL/I+tC+Bi8nvgYvp70GLye+yPrQvsf6UL/2obG/NNYHwF5OQcC8nILAAACAwOYUPMC8nALABi+nv+YUPL8GL6e+Bi+nvQAAAIAGL6e9Bi+nvuYUPL8GL6e/vJwCwOYUPMAAAIDAvJyCwF5OQcA01gfA9qGxv8f6UL/I+tC+Bi8nvgYvp70GLye+yPrQvsf6UL/2obG/NNYHwF5OQcC8nILA8HKKwMf6UMCdghfAyPrQvzTWh78GLye/yPrQvgYvp77I+tC+Bi8nvzTWh7/I+tC/nYIXwMf6UMDwcorAnYKXwCAaa8D2oTHAvJwCwOYUvL801oe/x/pQv+YUPL/H+lC/NNaHv+YUvL+8nALA9qExwCAaa8CdgpfAwsupwDTWh8A/NFbABi8nwLycAsDI+tC/9qGxvwYvp7/2obG/yPrQv7ycAsAGLyfAPzRWwDTWh8DCy6nAXk7BwNFYn8C8nILAPzRWwPahMcCdghfANNYHwLycAsA01gfAnYIXwPahMcA/NFbAvJyCwNFYn8BeTsHAcwrewOYUvMDRWJ/ANNaHwCAaa8DH+lDAXk5BwOYUPMBeTkHAx/pQwCAaa8A01ofA0VifwOYUvMBzCt7AAAAAwXMK3sBeTsHAwsupwJ2Cl8DwcorAvJyCwAAAgMC8nILA8HKKwJ2Cl8DCy6nAXk7BwHMK3sAAAADB", "arrayDtype": "float32", "validMaskBase64": "AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEB", "dataStrides": {"a2": 1, "a1": 15, "a0": 225}, "xAxisSpecs": [{"name": "a1", "label": "col: 15", "start": 0, "end": 15}, {"name": "a0", "label": "sign: 2", "start": 0, "end": 2}], "yAxisSpecs": [{"name": "a2", "label": "row: 15", "start": 0, "end": 15}], "slicedAxisSpecs": [], 

### Slicing and "scrubbing" with sliders

It's sometimes useful to only look at individual slices of a large array at a time, instead of viewing them all at once. In addition to the `columns` and `rows` arguments, arrayviz supports a `sliders` argument, which will display a slider for those axes and allow you to "scrub" through indices in it:

In [30]:
time = pz.nx.wrap(jnp.arange(100)).tag("time")
col = pz.nx.wrap(np.linspace(-2, 2, 15)).tag("col")
row = pz.nx.wrap(np.linspace(-2, 2, 15)).tag("row")

values_over_time = pz.nx.nmap(jax.nn.sigmoid)(
    0.05 * time - 2 - row - pz.nx.nmap(jnp.sin)(2 * col - 0.1 * time)
)

# Try sliding the slider:
pz.ts.render_array(
    values_over_time,
    columns=["col"],
    sliders=["time"],
)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "t4CjPl0C3D7p1xE/EiguP/GhOj/QhDY/M8shPwAAAD+7abw+XPaSPjG8ij7mr6M+VFDcPuT+ET+pPy4/3nWFPjoZuT51Wf8+BIEdP5VIKz/1sSY/Xz0QP/6s2z4W15s+ONhtPmeRXz62n4U+pmK5Pv6o/z6vmh0/knpWPhfUmD7ACds+n7kLPy5jGj/ycxU/jBX8PtvIuD5lY30+KK89PhOlMT57wlY+vBaZPqRX2z6D1As/5QYqPmkZeD5AL7g+sfTyPgR5CD/pXgM/ztjXPi6LmD6I5Eo+EWwVPu2CCz4CQyo+QI54PoZ4uD7LKvM+TUUFPuxiRj7k/5c+YfzOPu1v7D5oTuI+3i+1PpaZdz6zXSA+kQbpPQ0H2T10dgU+S8ZGPlNCmD6dMM8+nf7OPbGfHD6EpXY+DeusPka2yD7rDL8+h0mVPjT2RT6TyPo9gjW0PZuApz2HTc89KvIcPucZdz6aG60+Uo+fPT2z9D2KJkU+odmNPnQcpz6ePp4+x+ZxPnpFHD50X8I9cnOKPVt8gD2yzZ89PDn1PXuJRT4YBY4+34t0PXCDvT1mmRs+efVkPoOqiD7Gz4A+Ih9BProg9D2OmJU9E6RTPbAqRD1m7XQ9Wu69PW7rGz7ZQGU+Aoo6PYfEkT1OCfM9Ri42Ph7/Wz6Jgk4+QEMYPoMOvT0h/mQ9vBwhPdkwFT2A1To9uhiSPYOO8z3HbTY+zcgNPQwHXz2hL7w95zwPPqqlLj5MYCM+IKDtPX1okT3TgS493Ij0PHFG4jzcAg49MopfPdyZvD0jcQ8+2PHWPAnmKT0GuZA9egjfPYkNCT4qr/89Xt+3PbF3Xj2WigQ9wiK5PEI3qzyeStc8U0sqPakMkT3IXN89+pWiPFABAT1fZl09WkSsPV4S1T01S8Y9N1SNPVV3KT2Q0cg8ROmLPEFWgTym2aI8/U4BPZjoXT1Dh6w9CZJ1PCprwzxhpCg99TeEPTVepD3xr5g9Dx5YPWSsAD351Zc8ES1TPCInQzyc+HU80+HDPO4IKT1vbIQ9Hzs5PH26kzyOCgA9YvVJPdsQfD1h0Gk9zpAkPXvpwjwaQmU8VzQfPLUWEzzKiDk8rxSUPK9XAD3VRko9vpYLPDMHXzxa8sE8iKYZPTVdQD0TPDI9QdT5POpXkzwt4iw8QNvvO8eP3Tt40Qs8BpBfPChowjxD5Rk9ejqiPth21j7n3g4/NBUtP48tPD8//Do/JHcpP1WSCT/s9cw+cDidPu/ijj7RY6I+oMHWPhIHDz+eLy0/xFSEPk3ksz5qTvk+OFYcP9kELT/SrSs/CW0YP0+i7j4XCqs+c9F/PrjBZj5feYQ+Viq0Psyf+T7cchw/SopUPiAelD73I9U+H4EKP3VEHD+p0Bo/GnAGPz/Syj5qK4w+nPdMPh+yNz4XyVQ+UF2UPiJz1T4Gnwo/+mcoPhPcbz4zp7I+3IDwPpRuCj/z6gg/4WHoPiUPqT43DGI+exciPhx8ED54nCg+UkpwPj/xsj7hvPA+LPIDPg5kPz5XAJM+zZ7MPqdb8D6NU+0+ItXEPslmij7nujM+jpf9PaoL4T0IHQQ+cME/PgdDkz6g2Mw+f97MPdvUFj6Y6W0+YLmqPvV6zD6lkMk+S4qjPtz9Xj47OQ0+ip7EPWLdrT1II809ECIXPuJdbj7x7qo+QOGdPaVO6z3cvT0+R+OLPkGYqj775ac+XH6FPiUpMT4Zyds95l2XPbF4hT2dF549qsvrPU4gPj4lE4w+xetxPZcGtj0xeBU+dI9hPqHFiz7zXYk+K4lWPjUdCz5+sKk9HsFnPVTnSz2yQHI9MWq2PXrJFT484mE+vYE4PcTgiz0vGuk981EzPipcYT6kNF0+FRMqPlVj2D3lMoI9QaQwPSQmGz17wzg9Ay+MPcad6T2UlzM+0zgMPR7cVT0gRbQ9AuMMPs8mMz5PqS8+Qk8FPtL+pj3Q0kY9KS4GPdFl6zxTaww91VVWPeGttD04HA0+VI7UPOTSIj23f4o9OD7bPZK/DD5W4gk+oQ7PPb8WgD16PBc9SlLLPAUrsjyM29Q8yDAjPf/Rij1emts9mMSgPKgo9zwlt1M9Q0KpPSMF2z2taNY9+ZufPSuNQz0KaOU8gL2ZPOOdhjxg/6A8pLj3PAY3VD1ki6k9ms9yPC0juzxuKyE9ityBPfsUqT1ibaU9qJ90PZK3FD3wma08zCVoPBMnSzzSKHM8AZG7PBaOIT3WFYI9MiQ3PKpujTxwn/Q83ExGPRC5gT0MuX09U5k6PdCM4Tz+JYM81BEvPPsiGTzDZzc8IcKNPKc29Ty9pUY9kAIKPGd7VTz5M7k8V9UWPdIVRj0NpkE9ldQNPeapqjzW5UU8U+kDPGyx5jugNQo8+vlVPE6nuTzDGRc98tyhPuxs0T7cwws/Vp0rP1I5PT9k0T4/WnowP135Ej8Uid4+MteoPmAplD4sAKI+y7PRPszsCz9Uuis/9QGEPvEvrz5RCfM+1r4aP2QyLj94/y8/ggkgP0bUAD9de7s+5TSKPq7vbz4jIYQ+73GvPq9b8z4x3ho/HfxTPnjijz5JEM8+ZtgIPz6MHT8ggx8/y2IOP1ZN3T49/po+tqdePqZ0Pz6+MVQ+uh2QPtJfzz4L+Qg/SPEnPkl9aD6P/aw+dC7tPmHFCz9Q1Q0/KFP4PjVRuj4N5ns+zOAwPpfiFj72HSg+OuRoPntHrT61b+0+QpEDPlcoOT4x6o0+Dm3JPlkM8z4NNfc+tC/UPkzvmT5Cn0k+1uEKPuFk6z29tQM+P385PmAsjj6tq8k+BkPMPVawET41EmU+G8WnPjgTzz7kGdM+8sKxPiAKej5mTx8+yQPYPVAYtj2Mfcw98/cRPgKFZT7v/qc+YGadPW794j15RjY+rECJPkkArT6rv7A+3DKSPrsJSD7cEPk9GbOmPbDuiz2glJ09IHHjPTGnNj4vdIk+1itxPc5orz3OUA8+HQJdPqLsjT6fSZE+i4NsPqb+HT4SAME94bZ/PcvxVT0XdHE9vcSvPWGgDz4EW10+Ku03Pc2uhj2ZKN896n4vPm0WZT6Y7Wo+4Y48PkXt9j2Qg5Q9QTFDPZnjIj0VJTg95vaGPQKp3z2CyS8+q8YLPQfJTT3VXaw9kr8JPgpKNj6XNzs+LH4UPq5Kvz11TmM9ynAUPUZC9zyh8Qs9EDlOPcLDrD2//Ak+w9/TPEOZHD30S4Q9wzDWPbxTDz72YhM+jIXnPQIrkz1LNC09fSDhPLg2uzxxIdQ8lu8cPeCbhD0pk9Y9uz+gPDKe7TxuFEo9F0GlPVQt3z2eu+U9QgOzPYE1YT2KigM9b

You can even put sliders for multiple axes simultaneously, if you want:

In [31]:
row_wavelength = pz.nx.wrap(4 * jnp.arange(10) + 4).tag("row_wavelength")
col_wavelength = pz.nx.wrap(4 * jnp.arange(10) + 4).tag("col_wavelength")
col = pz.nx.wrap(np.arange(15)).tag("col")
row = pz.nx.wrap(np.arange(15)).tag("row")

values = (
    pz.nx.nmap(jnp.sin)(2 * np.pi * row / row_wavelength)
    * pz.nx.nmap(jnp.sin)(2 * np.pi * col / col_wavelength)
)

# Try sliding the slider:
pz.ts.render_array(
    values,
    columns=["col"],
    sliders=["row_wavelength", "col_wavelength"],
    axis_item_labels={
        "row_wavelength": [str(v) for v in row_wavelength.untag("row_wavelength").unwrap()],
        "col_wavelength": [str(v) for v in col_wavelength.untag("col_wavelength").unwrap()],
    }
)

ArrayvizRendering(html_src='<div class="arrayviz_container"><span class="loading_message">Rendering array...</span><treescope-run-here><script type="application/octet-stream">const parent = this.parentNode; const defns = this.getRootNode().host.defns; defns.runSoon(() => { const tpl = parent.querySelector(\'template.deferred_args\'); const config = JSON.parse( tpl.content.querySelector(\'script\').textContent ); tpl.remove(); defns.arrayviz.buildArrayvizFigure(parent, config); });</script></treescope-run-here><template class="deferred_args"><script type="application/json">{"info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAACAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAACAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAACAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAAAAAACAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAA

### Note: Memory usage

One caveat to using arrayviz: whenever you render an array, the *entire array* is serialized, saved directly into the notebook output cell, and then loaded into your browser's memory! That's true even if you use `sliders`; although only part of your array is visible, all of the data is there in the notebook and in your local browser, so that it can update the view when you move the slider.

This can sometimes be useful, since it means the visualization does not require Colab/IPython to be connected, and won't mess up any of your Python interpreter's state. On the other hand, it's easy to end up with very large Colab notebooks this way, and if you have many visualizations open, it can cause your web browser to bog down a bit. For a sense of scale here, a visualization of a `1000 x 1000` array adds about 5 megabytes to the size of your notebook. (Arrayviz will still happily render an array of that size, though!)

Given this, it's usually a good idea to avoid saving visualizations of very large arrays into the notebook. One way to do this is to turn on "Omit code cell output when saving this notebook" mode in Colab to avoid saving output from any cell.

## Using arrayviz in treescope

In JAX programs, NDArrays often occur as parts of large JAX-compatible data structures (PyTrees). If desired, you can use arrayviz to render these arrays inside a larger treescope rendering, by turning on automatic array visualization.

(You might want to read the [tutorial on treescope pretty-printing](treescope_prettyprinting.ipynb) before reading this section, if you haven't already.)

Let's start by registering treescope as the default pretty-printer:

In [32]:
pz.ts.register_as_default()

Ordinarily, treescope shows text representations of NDArrays, and lets you click to expand them:

In [33]:
IPython.display.display({
    "foo": pz.nx.wrap(jnp.arange(10)).tag("a") * pz.nx.wrap(jnp.arange(10)).tag("b"),
    "bar": np.sin(np.arange(100) * 0.1).reshape((10,10))
})

{'foo': <NamedArray int32(| a:10, b:10) [≥0, ≤81] zero:19 nonzero:81 (wrapping jax.Array)>,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933,  0.29552021,  0.38941834,
          0.47942554,  0.56464247,  0.64421769,  0.71735609,  0.78332691],
        [ 0.84147098,  0.89120736,  0.93203909,  0.96355819,  0.98544973,
          0.99749499,  0.9995736 ,  0.99166481,  0.97384763,  0.94630009],
        [ 0.90929743,  0.86320937,  0.8084964 ,  0.74570521,  0.67546318,
          0.59847214,  0.51550137,  0.42737988,  0.33498815,  0.23924933],
        [ 0.14112001,  0.04158066, -0.05837414, -0.15774569, -0.2555411 ,
         -0.35078323, -0.44252044, -0.52983614, -0.61185789, -0.68776616],
        [-0.7568025 , -0.81827711, -0.87157577, -0.91616594, -0.95160207,
         -0.97753012, -0.993691  , -0.99992326, -0.99616461, -0.98245261],
        [-0.95892427, -0.92581468, -0.88345466, -0.83226744, -0.77276449,
         -0.70554033, -0.63126664, -0.55068554, -0.46460218, -0.37387666],
        [-0.2794155 , -0.1821625 , -0.0830894 ,  0.0168139 ,  0.1165492 ,
          0.21511999,  0.31154136,  0.40484992,  0.49411335,  0.57843976],
        [ 0.6569866 ,  0.72896904,  0.79366786,  0.85043662,  0.8987081 ,
          0.93799998,  0.96791967,  0.98816823,  0.99854335,  0.99894134],
        [ 0.98935825,  0.96988981,  0.94073056,  0.90217183,  0.85459891,
          0.79848711,  0.7343971 ,  0.66296923,  0.58491719,  0.50102086],
        [ 0.41211849,  0.31909836,  0.22288991,  0.12445442,  0.02477543,
         -0.07515112, -0.17432678, -0.27176063, -0.36647913, -0.45753589]])}

You can enable array visualization by wrapping your `display` calls in an "autovisualizer" scope:

In [34]:
with pz.ts.active_autovisualizer.set_scoped(
    pz.ts.ArrayAutovisualizer()
):
  IPython.display.display({
      "foo": pz.nx.wrap(jnp.arange(10)).tag("a") * pz.nx.wrap(jnp.arange(10)).tag("b"),
      "bar": np.sin(np.arange(100) * 0.1).reshape((10,10))
  })

{'foo': <NamedArray int32(| a:10, b:10) [≥0, ≤81] zero:19 nonzero:81 (wrapping jax.Array)
   <Arrayviz rendering>
 >,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933,  0.29552021,  0.38941834,
          0.47942554,  0.56464247,  0.64421769,  0.71735609,  0.78332691],
        [ 0.84147098,  0.89120736,  0.93203909,  0.96355819,  0.98544973,
          0.99749499,  0.9995736 ,  0.99166481,  0.97384763,  0.94630009],
        [ 0.90929743,  0.86320937,  0.8084964 ,  0.74570521,  0.67546318,
          0.59847214,  0.51550137,  0.42737988,  0.33498815,  0.23924933],
        [ 0.14112001,  0.04158066, -0.05837414, -0.15774569, -0.2555411 ,
         -0.35078323, -0.44252044, -0.52983614, -0.61185789, -0.68776616],
        [-0.7568025 , -0.81827711, -0.87157577, -0.91616594, -0.95160207,
         -0.97753012, -0.993691  , -0.99992326, -0.99616461, -0.98245261],
        [-0.95892427, -0.92581468, -0.88345466, -0.83226744, -0.77276449,
         -0.70554033, -0.63126664, -0.55068554, -0.46460218, -0.37387666],
        [-0.2794155 , -0.1821625 , -0.0830894 ,  0.0168139 ,  0.1165492 ,
          0.21511999,  0.31154136,  0.40484992,  0.49411335,  0.57843976],
        [ 0.6569866 ,  0.72896904,  0.79366786,  0.85043662,  0.8987081 ,
          0.93799998,  0.96791967,  0.98816823,  0.99854335,  0.99894134],
        [ 0.98935825,  0.96988981,  0.94073056,  0.90217183,  0.85459891,
          0.79848711,  0.7343971 ,  0.66296923,  0.58491719,  0.50102086],
        [ 0.41211849,  0.31909836,  0.22288991,  0.12445442,  0.02477543,
         -0.07515112, -0.17432678, -0.27176063, -0.36647913, -0.45753589]])}

In Colab / IPython, you can optionally register the `%%autovisualize` cell magic that runs an IPython cell inside the autovisualizer scope:

In [35]:
pz.ts.register_autovisualize_magic()

In [36]:
%%autovisualize
{
  "foo": pz.nx.wrap(jnp.arange(10)).tag("a") * pz.nx.wrap(jnp.arange(10)).tag("b"),
  "bar": np.sin(np.arange(100) * 0.1).reshape((10,10))
}

{'foo': <NamedArray int32(| a:10, b:10) [≥0, ≤81] zero:19 nonzero:81 (wrapping jax.Array)
   <Arrayviz rendering>
 >,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933,  0.29552021,  0.38941834,
          0.47942554,  0.56464247,  0.64421769,  0.71735609,  0.78332691],
        [ 0.84147098,  0.89120736,  0.93203909,  0.96355819,  0.98544973,
          0.99749499,  0.9995736 ,  0.99166481,  0.97384763,  0.94630009],
        [ 0.90929743,  0.86320937,  0.8084964 ,  0.74570521,  0.67546318,
          0.59847214,  0.51550137,  0.42737988,  0.33498815,  0.23924933],
        [ 0.14112001,  0.04158066, -0.05837414, -0.15774569, -0.2555411 ,
         -0.35078323, -0.44252044, -0.52983614, -0.61185789, -0.68776616],
        [-0.7568025 , -0.81827711, -0.87157577, -0.91616594, -0.95160207,
         -0.97753012, -0.993691  , -0.99992326, -0.99616461, -0.98245261],
        [-0.95892427, -0.92581468, -0.88345466, -0.83226744, -0.77276449,
         -0.70554033, -0.63126664, -0.55068554, -0.46460218, -0.37387666],
        [-0.2794155 , -0.1821625 , -0.0830894 ,  0.0168139 ,  0.1165492 ,
          0.21511999,  0.31154136,  0.40484992,  0.49411335,  0.57843976],
        [ 0.6569866 ,  0.72896904,  0.79366786,  0.85043662,  0.8987081 ,
          0.93799998,  0.96791967,  0.98816823,  0.99854335,  0.99894134],
        [ 0.98935825,  0.96988981,  0.94073056,  0.90217183,  0.85459891,
          0.79848711,  0.7343971 ,  0.66296923,  0.58491719,  0.50102086],
        [ 0.41211849,  0.31909836,  0.22288991,  0.12445442,  0.02477543,
         -0.07515112, -0.17432678, -0.27176063, -0.36647913, -0.45753589]])}

Small arrays are shown in their entirety. Larger arrays are truncated to show only a subset of elements along each axis, to prevent visualizations from getting too large.

In [37]:
%%autovisualize
{
  "foo": pz.nx.wrap(jnp.arange(10_000)).tag("a") * pz.nx.wrap(jnp.arange(20)).tag("b"),
  "bar": np.sin(np.arange(250000) * 0.1).reshape((500,500))
}

{'foo': <NamedArray int32(| a:10000, b:20) [≥0, ≤189_981] zero:10_019 nonzero:189_981 (wrapping jax.Array)
   <Arrayviz rendering>
 >,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933, ..., -0.53582323,
         -0.44885398, -0.35739993],
        [-0.26237485, -0.16472822, -0.06543567, ..., -0.73858223,
         -0.66758835, -0.58992416],
        [-0.50636564, -0.41774768, -0.32495573, ..., -0.88959028,
         -0.83954619, -0.78111362],
        ...,
        [ 0.0021101 ,  0.10193276,  0.20073693, ..., -0.53404042,
         -0.44696738, -0.3554284 ],
        [-0.26033809, -0.16264657, -0.06332994, ..., -0.73715803,
         -0.66601583, -0.58821903],
        [-0.50454493, -0.41582959, -0.32295942, ..., -0.88862449,
         -0.83839792, -0.77979436]])}

If you want to enable array visualization all of the time, you can set up an automatic visualizer for your whole session:

In [38]:
# Makes it possible to set penzai's contextual configuration options interactively
pz.enable_interactive_context()

# (Later you can call `pz.disable_interactive_context()` to
# reset all of them to their default values.)

In [39]:
pz.ts.active_autovisualizer.set_interactive(
    pz.ts.ArrayAutovisualizer()
)

Now every value you display, or return from a cell, will be rendered with arrayviz:

In [40]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

`ArrayAutovisualizer` takes some configuration options that allow you to configure the summarization threshold and number of edge items visualized; see `help(ArrayAutovisualizer)` for more info.

If you've enabled automatic array visualization, you can disable it in a specific cell using `%%autovisualize None`:

In [41]:
%%autovisualize None
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## Custom treescope visualizations with autovisualizers and figure inlining

Automatic array visualization is a special case of a more general treescope feature, which lets you render arbitrary figures at arbitrary points in pretty-printed PyTrees. To customize automatic visualization, you define an autovisualizer function, with the following signature:
```python
def autovisualizer_fn(
    value: Any,
    path: tuple[Any, ...] | None,
) -> pz.ts.IPythonVisualization | pz.ts.ChildAutovisualizer | None:
  ...
```
This function will be called on every subtree of the rendered tree, and can return `pz.ts.IPythonVisualization(some_figure)` to replace the subtree with a visualization, or `None` to process the subtree normally. (It can also return `pz.ts.ChildAutovisualizer` if the subtree should be rendered with a different autovisualizer.)

For instance, we can write an autovisualizer that always formats arrays in continuous mode:

In [42]:
def my_continuous_autovisualizer(
    value: Any,
    path: tuple[Any, ...] | None,
):
  if isinstance(value, (np.ndarray, pz.nx.NamedArray)):
    return pz.ts.IPythonVisualization(
        pz.ts.render_array(value, continuous=True, around_zero=False))

In [43]:
with pz.ts.active_autovisualizer.set_scoped(
    my_continuous_autovisualizer
):
  IPython.display.display({
      "foo": pz.nx.wrap(jnp.arange(10)).tag("a") * pz.nx.wrap(jnp.arange(10)).tag("b"),
      "bar": np.sin(np.arange(100) * 0.1).reshape((10,10))
  })

{'foo': <NamedArray int32(| a:10, b:10) [≥0, ≤81] zero:19 nonzero:81 (wrapping jax.Array)>
 #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
 <rich HTML visualization>
 #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
 ,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933,  0.29552021,  0.38941834,
          0.47942554,  0.56464247,  0.64421769,  0.71735609,  0.78332691],
        [ 0.84147098,  0.89120736,  0.93203909,  0.96355819,  0.98544973,
          0.99749499,  0.9995736 ,  0.99166481,  0.97384763,  0.94630009],
        [ 0.90929743,  0.86320937,  0.8084964 ,  0.74570521,  0.67546318,
          0.59847214,  0.51550137,  0.42737988,  0.33498815,  0.23924933],
        [ 0.14112001,  0.04158066, -0.05837414, -0.15774569, -0.2555411 ,
         -0.35078323, -0.44252044, -0.52983614, -0.61185789, -0.68776616],
        [-0.7568025 , -0.81827711, -0.87157577, -0.91616594, -0.95160207,
         -0.97753012, -0.993691  , -0.99992326, -0.99616461, -0.98245261],
        [-0.95892427, -0.92581468, -0.88345466, -0.83226744, -0.77276449,
         -0.70554033, -0.63126664, -0.55068554, -0.46460218, -0.37387666],
        [-0.2794155 , -0.1821625 , -0.0830894 ,  0.0168139 ,  0.1165492 ,
          0.21511999,  0.31154136,  0.40484992,  0.49411335,  0.57843976],
        [ 0.6569866 ,  0.72896904,  0.79366786,  0.85043662,  0.8987081 ,
          0.93799998,  0.96791967,  0.98816823,  0.99854335,  0.99894134],
        [ 0.98935825,  0.96988981,  0.94073056,  0.90217183,  0.85459891,
          0.79848711,  0.7343971 ,  0.66296923,  0.58491719,  0.50102086],
        [ 0.41211849,  0.31909836,  0.22288991,  0.12445442,  0.02477543,
         -0.07515112, -0.17432678, -0.27176063, -0.36647913, -0.45753589]])}

Or, add additional metadata:

In [44]:
def my_verbose_autovisualizer(
    value: Any,
    path: tuple[Any, ...] | None,
):
  if isinstance(value, (np.ndarray, pz.nx.NamedArrayBase)):
    if isinstance(value, pz.nx.NamedArrayBase):
      size = value.data_array.size
    else:
      size = value.size
    return pz.ts.IPythonVisualization(
        pz.ts.inline(
            "Hello world!\n",
            pz.ts.render_array(value),
            f"\nThis array contains {size} elements and has Python id {id(value):,}, which you could tokenize as  ",
            pz.ts.integer_digitbox(id(value) // 1000000000000),
            "   ", pz.ts.integer_digitbox((id(value) // 1000000000) % 1000),
            "   ", pz.ts.integer_digitbox((id(value) // 1000000) % 1000),
            "   ", pz.ts.integer_digitbox((id(value) // 1000) % 1000),
            "   ", pz.ts.integer_digitbox(id(value) % 1000),
            f"\nThe path to this node is {path}",
        )
    )

In [45]:
with pz.ts.active_autovisualizer.set_scoped(
    my_verbose_autovisualizer
):
  IPython.display.display({
      "foo": pz.nx.wrap(jnp.arange(10)).tag("a") * pz.nx.wrap(jnp.arange(10)).tag("b"),
      "bar": np.sin(np.arange(150) * 0.1).reshape((15,10))
  })

{'foo': <NamedArray int32(| a:10, b:10) [≥0, ≤81] zero:19 nonzero:81 (wrapping jax.Array)>
 #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
 <rich HTML visualization>
 #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
 ,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933,  0.29552021,  0.38941834,
          0.47942554,  0.56464247,  0.64421769,  0.71735609,  0.78332691],
        [ 0.84147098,  0.89120736,  0.93203909,  0.96355819,  0.98544973,
          0.99749499,  0.9995736 ,  0.99166481,  0.97384763,  0.94630009],
        [ 0.90929743,  0.86320937,  0.8084964 ,  0.74570521,  0.67546318,
          0.59847214,  0.51550137,  0.42737988,  0.33498815,  0.23924933],
        [ 0.14112001,  0.04158066, -0.05837414, -0.15774569, -0.2555411 ,
         -0.35078323, -0.44252044, -0.52983614, -0.61185789, -0.68776616],
        [-0.7568025 , -0.81827711, -0.87157577, -0.91616594, -0.95160207,
         -0.97753012, -0.993691  , -0.99992326, -0.99616461, -0.98245261],
        [-0.95892427, -0.92581468, -0.88345466, -0.83226744, -0.77276449,
         -0.70554033, -0.63126664, -0.55068554, -0.46460218, -0.37387666],
        [-0.2794155 , -0.1821625 , -0.0830894 ,  0.0168139 ,  0.1165492 ,
          0.21511999,  0.31154136,  0.40484992,  0.49411335,  0.57843976],
        [ 0.6569866 ,  0.72896904,  0.79366786,  0.85043662,  0.8987081 ,
          0.93799998,  0.96791967,  0.98816823,  0.99854335,  0.99894134],
        [ 0.98935825,  0.96988981,  0.94073056,  0.90217183,  0.85459891,
          0.79848711,  0.7343971 ,  0.66296923,  0.58491719,  0.50102086],
        [ 0.41211849,  0.31909836,  0.22288991,  0.12445442,  0.02477543,
         -0.07515112, -0.17432678, -0.27176063, -0.36647913, -0.45753589],
        [-0.54402111, -0.62507065, -0.69987469, -0.76768581, -0.82782647,
         -0.87969576, -0.92277542, -0.95663502, -0.98093623, -0.99543625],
        [-0.99999021, -0.99455259, -0.97917773, -0.95401925, -0.91932853,
         -0.87545217, -0.82282859, -0.76198358, -0.69352508, -0.61813711],
        [-0.53657292, -0.44964746, -0.35822928, -0.26323179, -0.16560418,
         -0.0663219 ,  0.03362305,  0.13323204,  0.23150983,  0.32747444],
        [ 0.42016704,  0.50866146,  0.59207351,  0.66956976,  0.74037589,
          0.80378443,  0.85916181,  0.90595474,  0.94369567,  0.9720075 ],
        [ 0.99060736,  0.99930939,  0.99802665,  0.98677196,  0.96565778,
          0.93489506,  0.89479117,  0.84574683,  0.78825207,  0.72288135]])}

You can even render values using an external plotting library like `plotly`!

Treescope can inline any type of figure that has a rich HTML representation (specifically, any object that defines the magic `_repr_html_` method expected by Colab's IPython kernel.)

In [46]:
import plotly.express as px

In [47]:
def my_plotly_autovisualizer(
    value: Any,
    path: tuple[Any, ...] | None,
):
  if isinstance(value, (np.ndarray, jax.Array)):
    return pz.ts.IPythonVisualization(
        px.histogram(
            value.flatten(),
            width=400, height=200
        ).update_layout(
            margin=dict(l=20, r=20, t=20, b=20)
        )
    )

In [48]:
with pz.ts.active_autovisualizer.set_scoped(
    my_plotly_autovisualizer
):
  IPython.display.display({
      "foo": pz.nx.wrap(jnp.arange(10)).tag("a") * pz.nx.wrap(jnp.arange(10)).tag("b"),
      "bar": np.sin(np.arange(150) * 0.1).reshape((15,10))
  })

{'foo': <NamedArray int32(| a:10, b:10) [≥0, ≤81] zero:19 nonzero:81 (wrapping jax.Array)>,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933,  0.29552021,  0.38941834,
          0.47942554,  0.56464247,  0.64421769,  0.71735609,  0.78332691],
        [ 0.84147098,  0.89120736,  0.93203909,  0.96355819,  0.98544973,
          0.99749499,  0.9995736 ,  0.99166481,  0.97384763,  0.94630009],
        [ 0.90929743,  0.86320937,  0.8084964 ,  0.74570521,  0.67546318,
          0.59847214,  0.51550137,  0.42737988,  0.33498815,  0.23924933],
        [ 0.14112001,  0.04158066, -0.05837414, -0.15774569, -0.2555411 ,
         -0.35078323, -0.44252044, -0.52983614, -0.61185789, -0.68776616],
        [-0.7568025 , -0.81827711, -0.87157577, -0.91616594, -0.95160207,
         -0.97753012, -0.993691  , -0.99992326, -0.99616461, -0.98245261],
        [-0.95892427, -0.92581468, -0.88345466, -0.83226744, -0.77276449,
         -0.70554033, -0.63126664, -0.55068554, -0.46460218, -0.37387666],
        [-0.2794155 , -0.1821625 , -0.0830894 ,  0.0168139 ,  0.1165492 ,
          0.21511999,  0.31154136,  0.40484992,  0.49411335,  0.57843976],
        [ 0.6569866 ,  0.72896904,  0.79366786,  0.85043662,  0.8987081 ,
          0.93799998,  0.96791967,  0.98816823,  0.99854335,  0.99894134],
        [ 0.98935825,  0.96988981,  0.94073056,  0.90217183,  0.85459891,
          0.79848711,  0.7343971 ,  0.66296923,  0.58491719,  0.50102086],
        [ 0.41211849,  0.31909836,  0.22288991,  0.12445442,  0.02477543,
         -0.07515112, -0.17432678, -0.27176063, -0.36647913, -0.45753589],
        [-0.54402111, -0.62507065, -0.69987469, -0.76768581, -0.82782647,
         -0.87969576, -0.92277542, -0.95663502, -0.98093623, -0.99543625],
        [-0.99999021, -0.99455259, -0.97917773, -0.95401925, -0.91932853,
         -0.87545217, -0.82282859, -0.76198358, -0.69352508, -0.61813711],
        [-0.53657292, -0.44964746, -0.35822928, -0.26323179, -0.16560418,
         -0.0663219 ,  0.03362305,  0.13323204,  0.23150983,  0.32747444],
        [ 0.42016704,  0.50866146,  0.59207351,  0.66956976,  0.74037589,
          0.80378443,  0.85916181,  0.90595474,  0.94369567,  0.9720075 ],
        [ 0.99060736,  0.99930939,  0.99802665,  0.98677196,  0.96565778,
          0.93489506,  0.89479117,  0.84574683,  0.78825207,  0.72288135]])}

You can also pass custom visualizers to the `%%autovisualize` magic to let it handle the `set_scoped` boilerplate for you:

In [49]:
%%autovisualize my_plotly_autovisualizer
{
  "foo": pz.nx.wrap(jnp.arange(10)).tag("a") * pz.nx.wrap(jnp.arange(10)).tag("b"),
  "bar": np.sin(np.arange(150) * 0.1).reshape((15,10))
}

{'foo': <NamedArray int32(| a:10, b:10) [≥0, ≤81] zero:19 nonzero:81 (wrapping jax.Array)>,
 'bar': array([[ 0.        ,  0.09983342,  0.19866933,  0.29552021,  0.38941834,
          0.47942554,  0.56464247,  0.64421769,  0.71735609,  0.78332691],
        [ 0.84147098,  0.89120736,  0.93203909,  0.96355819,  0.98544973,
          0.99749499,  0.9995736 ,  0.99166481,  0.97384763,  0.94630009],
        [ 0.90929743,  0.86320937,  0.8084964 ,  0.74570521,  0.67546318,
          0.59847214,  0.51550137,  0.42737988,  0.33498815,  0.23924933],
        [ 0.14112001,  0.04158066, -0.05837414, -0.15774569, -0.2555411 ,
         -0.35078323, -0.44252044, -0.52983614, -0.61185789, -0.68776616],
        [-0.7568025 , -0.81827711, -0.87157577, -0.91616594, -0.95160207,
         -0.97753012, -0.993691  , -0.99992326, -0.99616461, -0.98245261],
        [-0.95892427, -0.92581468, -0.88345466, -0.83226744, -0.77276449,
         -0.70554033, -0.63126664, -0.55068554, -0.46460218, -0.37387666],
        [-0.2794155 , -0.1821625 , -0.0830894 ,  0.0168139 ,  0.1165492 ,
          0.21511999,  0.31154136,  0.40484992,  0.49411335,  0.57843976],
        [ 0.6569866 ,  0.72896904,  0.79366786,  0.85043662,  0.8987081 ,
          0.93799998,  0.96791967,  0.98816823,  0.99854335,  0.99894134],
        [ 0.98935825,  0.96988981,  0.94073056,  0.90217183,  0.85459891,
          0.79848711,  0.7343971 ,  0.66296923,  0.58491719,  0.50102086],
        [ 0.41211849,  0.31909836,  0.22288991,  0.12445442,  0.02477543,
         -0.07515112, -0.17432678, -0.27176063, -0.36647913, -0.45753589],
        [-0.54402111, -0.62507065, -0.69987469, -0.76768581, -0.82782647,
         -0.87969576, -0.92277542, -0.95663502, -0.98093623, -0.99543625],
        [-0.99999021, -0.99455259, -0.97917773, -0.95401925, -0.91932853,
         -0.87545217, -0.82282859, -0.76198358, -0.69352508, -0.61813711],
        [-0.53657292, -0.44964746, -0.35822928, -0.26323179, -0.16560418,
         -0.0663219 ,  0.03362305,  0.13323204,  0.23150983,  0.32747444],
        [ 0.42016704,  0.50866146,  0.59207351,  0.66956976,  0.74037589,
          0.80378443,  0.85916181,  0.90595474,  0.94369567,  0.9720075 ],
        [ 0.99060736,  0.99930939,  0.99802665,  0.98677196,  0.96565778,
          0.93489506,  0.89479117,  0.84574683,  0.78825207,  0.72288135]])}